## Gerando arquivos para NER-NestedClinBr

Deixar sempre o arquivo de test para fazer o teste completo (com o pipeline todo)
1o. -> passar pelo modelo de NER normal (binario), pegar entidades nivel externo
2o. -> novo modelo de NER para entidades internas

Descontinuas: tentar RBERT - relação entre entidades... pensar em como fazer.. pra nao ficar pesado... se estiver desbalanceado, dar menor peso para label O no crossentropy loss

In [1]:
import os
from pathlib import Path
import re
import pickle
import random


In [2]:
def getTiposEntidade():
    return ['Problema','Teste','Tratamento','Anatomia']

In [3]:
def replaceWhiteSpaces(str):
    return re.sub('\s{2,}',' ',str)

In [4]:
def save_obj(name, obj):
    existeDir = os.path.exists('obj')
    if not existeDir:
        os.makedirs('obj')
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name):
    existeDir = os.path.exists('obj')
    if not existeDir:
        os.makedirs('obj')
    try:
        with open('obj/' + name + '.pkl', 'rb') as f:
            return pickle.load(f)
    except:
        return None


In [5]:
pastasCorpus = [r'C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\preProcessamento\corpus\test',r'C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\preProcessamento\corpus\train']
#pastasCorpus = [r'corpus\test']
#pastasCorpus = [r'corpus\TESTE']

In [6]:
import re

s = 'Make the World a 2.4Better Place2.0'
pattern = r'([0-9])\.([0-9])'
replacement = r'\1y\2'
html = re.sub(pattern, replacement, s)

print(html)

Make the World a 2y4Better Place2y0


In [30]:
# com todas entidades corretamente (descontinuas)
def getDicSentencesGabarito(pastaCorpus):
    #devePrintar=True
    devePrintar=False
    dic_sentences = {}
    numMaxTokensPorFrase=0 # numero tokens da maior frase
    num=0
    listaEntidades=[]
    frasesComDescontinuas=[]
    for filename in os.listdir(pastaCorpus):
        f = os.path.join(pastaCorpus, filename)
        if os.path.isfile(f):
            fileNameSemExtensao=os.path.splitext(filename)[0]
            if devePrintar:
                print('\n\n--fileName (sem extensao):--', fileNameSemExtensao)
                pass
            extension = os.path.splitext(filename)[1][1:]
            if extension=='ann':
                # tokens da frase
                fileTxt = open(os.path.join(pastaCorpus, fileNameSemExtensao)+'.txt', "r", encoding='utf-8')
                linha=fileTxt.readlines()
                fileTxt.close()
                if devePrintar:
                    print('linha:', linha)
                    pass
                frases=[]
                allFrasesString=''
                numL=0
                for l in linha:
                    numL=numL+1
                    allFrasesString = allFrasesString+l
                    if numL==1: # descarta primeira frase, que é data de criação do doc
                        #print('descartando frase:', l)
                        continue
                    if l.strip() and l.strip()!='\n':
                        #print('l:', l)
                        pattern = r'([0-9])\.([0-9])'
                        replacement = r'\1==\2'
                        novoL = re.sub(pattern, replacement, l.strip())
                        l2 = novoL.split('.') # quebrando frases
                        for l3 in l2:
                            if l3.strip() and l3.strip()!='\n':
                                novaFrase = l3.replace('\n','').replace('==','.').strip()+'.'
                                frases.append(novaFrase)
                #print('frases:', frases)
                # para cada frase
                # para tokenizar nesses tokens
                #print('allFrasesString:', allFrasesString)
                frasesTokens={}
                #numCaracteresTotal=42 # primeira frase ignorada
                numIndiceAnterior=0
                for frase in frases:
                    tokens=[]
                    frase2 = frase.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                    frase2 = replaceWhiteSpaces(frase2)
                    frase2 = frase2.split()
                    for numtoken, token in enumerate(frase2):
                        if devePrintar:
                            print('token:', token)
                            print('numIndiceAnterior:', numIndiceAnterior)
                        if token!='.':
                            numCaracteresTotal = allFrasesString.find(token, numIndiceAnterior, len(allFrasesString))
                        else:
                            numCaracteresTotal=numIndiceAnterior+1
                        #tokens.append([token,numtoken])
                        tokens.append([token,numtoken, numCaracteresTotal])
                        numIndiceAnterior = numCaracteresTotal+len(token)-1
                        if numMaxTokensPorFrase<len(tokens):
                            numMaxTokensPorFrase = len(tokens)
                    frasesTokens[frase]=tokens
                    #linhaTokens=linha.copy()    
                if devePrintar:
                    print('frasesTokens:', frasesTokens)
                # agora, as entidades
                fileAnn = open(f, "r", encoding='utf-8')
                linha=fileAnn.readlines()
                fileAnn.close()

                dicEntidades={}
                dicEntidadesDescontinuas={}
                listaEntidadesDescontinuas={}
                numIndiceDesc=0
                for entidade_linha in linha:
                    if ';' not in entidade_linha: # nao é descontinua
                        entidade = entidade_linha.split('\t')
                        tipo_entidade = entidade[1]
                        inicio, fim = tipo_entidade.split()[1:3]
                        tipo_entidade = tipo_entidade.split()[0]
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        dicEntidades[(int(inicio), int(fim))]=[tipo_entidade, replaceWhiteSpaces(termos_entidade)]
                    else:
                        frasesComDescontinuas.append(num)
                        #print('descontinua, linha: {}, num: {}'.format(linha, num))
                        #print('descontinua, file: {}, num: {}'.format(filename, num))
                        
                        # grava com os indices corretoss
                        entidade = entidade_linha.split('\t')
                        # ex T10	Problema 244 252;279 306	dispneia aos mdoeardos-leves esforço
                        #Problema 244 252;279 306
                        entidade_temp=entidade[1].split(';')
                        entidade1=entidade_temp[0]
                        tipo_entidade = entidade1
                        inicio1, fim1 = tipo_entidade.split()[1:3]
                        tipo_entidade_string = tipo_entidade.split()[0]
                        # mandar só os termos referentes...
                        tamTermo1=int(fim1)-int(inicio1)
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade=termos_entidade[:tamTermo1]
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        #print('aaaaaaaaaaaaaaaaa')
                        
                        dicEntidadesDescontinuas[(int(inicio1), int(fim1))]=[tipo_entidade_string, replaceWhiteSpaces(termos_entidade)]
                        #print("(int(inicio1), int(fim1)):", (int(inicio1), int(fim1)))
                        #print("tipo_entidade_string, replaceWhiteSpaces(termos_entidade):", tipo_entidade_string, replaceWhiteSpaces(termos_entidade))
                        
                        entidade2=entidade_temp[1]
                        inicio2, fim2 = entidade2.split()[0:2]
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade=termos_entidade[tamTermo1:len(termos_entidade)]
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        dicEntidadesDescontinuas[(int(inicio2), int(fim2))]=[tipo_entidade_string, replaceWhiteSpaces(termos_entidade)]
                        #print("(int(inicio2), int(fim2)):", (int(inicio2), int(fim2)))
                        #print("tipo_entidade_string, replaceWhiteSpaces(termos_entidade):", tipo_entidade_string, replaceWhiteSpaces(termos_entidade))
                        listaEntidadesDescontinuas[numIndiceDesc] = dicEntidadesDescontinuas
                        dicEntidadesDescontinuas={}
                        numIndiceDesc=numIndiceDesc+1
                        
                        
                print('--listaEntidadesDescontinuas:--', listaEntidadesDescontinuas)
                        
                print('frasesTokens:', frasesTokens)
                indicesDic = sorted(dicEntidades.keys(), key = lambda item: item[0])
                #indicesDicDescontinuas = sorted(dicEntidadesDescontinuas.keys(), key = lambda item: item[0])
                listaIndicesJaUsados = []
                #list_students.sort(key = lambda x: x[1])   #index 1 means second element
                #print('indicesDicDescontinuas:', indicesDicDescontinuas)
                #print('dicEntidades:', dicEntidades)
                
                for key, value in frasesTokens.items():
                    print('key:', key)
                    print('value:', value)
                    for i in indicesDic:
                        tipo_entidade,termos_entidade = dicEntidades[i]
                        ##key (i) = indices old a comparar
                        #print('i:', i)
                        for token in value:
                            #print('token:', token)
                            if i[0]==token[2]:
                                novo_inicio, novo_fim = [token[1],token[1]+len(termos_entidade.split())]
                                novos_indices=[]
                                for k in range(novo_inicio,novo_fim):
                                    novos_indices.append(k)
                                listaEntidades.append([termos_entidade, novos_indices, tipo_entidade])
                                print("[termos_entidade, novos_indices, tipo_entidade]:", [termos_entidade, novos_indices, tipo_entidade])
                            else:
                                #print('else, i[0]:',i[0])
                                pass
                    # agora, descontinuas
                    for keyD, valueD in listaEntidadesDescontinuas.items():
                        print('valueD>', valueD)
                        listTemp= list(valueD)
                        indice1=listTemp[0]
                        indice2=listTemp[1]
                        tipo_entidade1,termos_entidade1 = valueD[indice1]
                        tipo_entidade2,termos_entidade2 = valueD[indice2]
                        print('indice1:{},tipo_entidade1:{},termos_entidade1:{},indice2:{},tipo_entidade2:{},termos_entidade2:{}'.format(indice1, tipo_entidade1,termos_entidade1,indice2, tipo_entidade2,termos_entidade2))
                        achou=0
                        token1=''
                        for token in value:
                            #print('token:', token)
                            if indice1[0]==token[2]: # indice igual na primeira palavra
                                achou=1
                                token1 = token
                            if achou==1:
                                if indice2[0]==token[2]: # segunda palavra bate tb
                                    novo_inicio1, novo_fim1 = [token1[1],token1[1]+len(termos_entidade1.split())]
                                    novo_inicio2, novo_fim2 = [token[1],token[1]+len(termos_entidade2.split())]
                                    novos_indices=[]
                                    for k in range(novo_inicio1,novo_fim1):
                                        novos_indices.append(k)
                                    for k in range(novo_inicio2,novo_fim2):
                                        novos_indices.append(k)
                                    listaEntidades.append([termos_entidade1+' '+termos_entidade2, novos_indices, tipo_entidade1])
                                    print('[termos_entidade1+' '+termos_entidade2, novos_indices, tipo_entidade1]', [termos_entidade1+' '+termos_entidade2, novos_indices, tipo_entidade1])
                        
                    if len(value)>0:
                        #print('incluindo:', key)
                        dic_sentences[num]=[value, listaEntidades]
                        listaEntidades=[]
                        num=num+1 

        #print(num)
        #if num>318:
        #    break

        if num>5:
            break
    print('numMaxTokensPorFrase:', numMaxTokensPorFrase)
    return dic_sentences, frasesComDescontinuas
            
dic_sentencesTest, frasesComDescontinuasTest = getDicSentencesGabarito(pastasCorpus[0])


--listaEntidadesDescontinuas:-- {0: {(695, 699): ['Problema', 'MMII'], (704, 709): ['Problema', 'edema']}}
frasesTokens: {'Lucas, 74 anos.': [['Lucas', 0, 43], [',', 1, 48], ['74', 2, 50], ['anos', 3, 53], ['.', 4, 57]], 'Em acompanhamento no ambualtorio há 5 anos por FA, uso de marevan 5mg 1 x ao dia.': [['Em', 0, 59], ['acompanhamento', 1, 62], ['no', 2, 77], ['ambualtorio', 3, 80], ['há', 4, 92], ['5', 5, 95], ['anos', 6, 97], ['por', 7, 102], ['FA', 8, 106], [',', 9, 108], ['uso', 10, 110], ['de', 11, 114], ['marevan', 12, 117], ['5mg', 13, 125], ['1', 14, 129], ['x', 15, 131], ['ao', 16, 133], ['dia', 17, 136], ['.', 18, 139]], 'Comorbidades: DM há 10 anos em uso de metformina 850mg 3 cp/dia, acarbose 1 cp/dia e glicazida 60mg 2 cp/dia e insulina (24 - 0 - 24).': [['Comorbidades', 0, 142], [':', 1, 154], ['DM', 2, 156], ['há', 3, 159], ['10', 4, 162], ['anos', 5, 165], ['em', 6, 170], ['uso', 7, 173], ['de', 8, 177], ['metformina', 9, 180], ['850mg', 10, 191], ['3', 11, 197], ['cp

In [31]:
dic_sentencesTest

{0: [[['Lucas', 0, 43],
   [',', 1, 48],
   ['74', 2, 50],
   ['anos', 3, 53],
   ['.', 4, 57]],
  []],
 1: [[['Em', 0, 59],
   ['acompanhamento', 1, 62],
   ['no', 2, 77],
   ['ambualtorio', 3, 80],
   ['há', 4, 92],
   ['5', 5, 95],
   ['anos', 6, 97],
   ['por', 7, 102],
   ['FA', 8, 106],
   [',', 9, 108],
   ['uso', 10, 110],
   ['de', 11, 114],
   ['marevan', 12, 117],
   ['5mg', 13, 125],
   ['1', 14, 129],
   ['x', 15, 131],
   ['ao', 16, 133],
   ['dia', 17, 136],
   ['.', 18, 139]],
  [['FA', [8], 'Problema'], ['marevan 5mg', [12, 13], 'Tratamento']]],
 2: [[['Comorbidades', 0, 142],
   [':', 1, 154],
   ['DM', 2, 156],
   ['há', 3, 159],
   ['10', 4, 162],
   ['anos', 5, 165],
   ['em', 6, 170],
   ['uso', 7, 173],
   ['de', 8, 177],
   ['metformina', 9, 180],
   ['850mg', 10, 191],
   ['3', 11, 197],
   ['cp', 12, 199],
   ['/', 13, 201],
   ['dia', 14, 202],
   [',', 15, 205],
   ['acarbose', 16, 207],
   ['1', 17, 216],
   ['cp', 18, 218],
   ['/', 19, 220],
   ['dia', 20

In [17]:
'''
def getDicSentences(pastaCorpus):
    #devePrintar=True
    devePrintar=False
    dic_sentences = {}
    numMaxTokensPorFrase=0 # numero tokens da maior frase
    num=0
    listaEntidades=[]
    frasesComDescontinuas=[]
    for filename in os.listdir(pastaCorpus):
        f = os.path.join(pastaCorpus, filename)
        if os.path.isfile(f):
            fileNameSemExtensao=os.path.splitext(filename)[0]
            if devePrintar:
                print('\n\n--fileName (sem extensao):--', fileNameSemExtensao)
                pass
            extension = os.path.splitext(filename)[1][1:]
            if extension=='ann':
                # tokens da frase
                fileTxt = open(os.path.join(pastaCorpus, fileNameSemExtensao)+'.txt', "r", encoding='utf-8')
                linha=fileTxt.readlines()
                fileTxt.close()
                if devePrintar:
                    print('linha:', linha)
                    pass
                frases=[]
                allFrasesString=''
                numL=0
                for l in linha:
                    numL=numL+1
                    allFrasesString = allFrasesString+l
                    if numL==1: # descarta primeira frase, que é data de criação do doc
                        #print('descartando frase:', l)
                        continue
                    if l.strip() and l.strip()!='\n':
                        #print('l:', l)
                        pattern = r'([0-9])\.([0-9])'
                        replacement = r'\1==\2'
                        novoL = re.sub(pattern, replacement, l.strip())
                        l2 = novoL.split('.') # quebrando frases
                        for l3 in l2:
                            if l3.strip() and l3.strip()!='\n':
                                novaFrase = l3.replace('\n','').replace('==','.').strip()+'.'
                                frases.append(novaFrase)
                #print('frases:', frases)
                # para cada frase
                # para tokenizar nesses tokens
                #print('allFrasesString:', allFrasesString)
                frasesTokens={}
                #numCaracteresTotal=42 # primeira frase ignorada
                numIndiceAnterior=0
                for frase in frases:
                    tokens=[]
                    frase2 = frase.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                    frase2 = replaceWhiteSpaces(frase2)
                    frase2 = frase2.split()
                    for numtoken, token in enumerate(frase2):
                        if devePrintar:
                            print('token:', token)
                            print('numIndiceAnterior:', numIndiceAnterior)
                        if token!='.':
                            numCaracteresTotal = allFrasesString.find(token, numIndiceAnterior, len(allFrasesString))
                        else:
                            numCaracteresTotal=numIndiceAnterior+1
                        #tokens.append([token,numtoken])
                        tokens.append([token,numtoken, numCaracteresTotal])
                        numIndiceAnterior = numCaracteresTotal+len(token)-1
                        if numMaxTokensPorFrase<len(tokens):
                            numMaxTokensPorFrase = len(tokens)
                    frasesTokens[frase]=tokens
                    #linhaTokens=linha.copy()    
                if devePrintar:
                    print('frasesTokens:', frasesTokens)
                # agora, as entidades
                fileAnn = open(f, "r", encoding='utf-8')
                linha=fileAnn.readlines()
                fileAnn.close()

                dicEntidades={}
                for entidade_linha in linha:
                    if ';' not in entidade_linha: # tem descontinua?
                        entidade = entidade_linha.split('\t')
                        tipo_entidade = entidade[1]
                        inicio, fim = tipo_entidade.split()[1:3]
                        tipo_entidade = tipo_entidade.split()[0]
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        dicEntidades[(int(inicio), int(fim))]=[tipo_entidade, replaceWhiteSpaces(termos_entidade)]
                    else:
                        frasesComDescontinuas.append(num)
                        #print('descontinua, linha: {}, num: {}'.format(linha, num))
                        print('descontinua, file: {}, num: {}'.format(filename, num))
                        entidade = entidade_linha.split('\t')
                        # ex T10	Problema 244 252;279 306	dispneia aos mdoeardos-leves esforço
                        #Problema 244 252;279 306
                        entidade_temp=entidade[1].split(';')
                        entidade1=entidade_temp[0]
                        tipo_entidade = entidade1
                        inicio1, fim1 = tipo_entidade.split()[1:3]
                        tipo_entidade_string = tipo_entidade.split()[0]
                        # mandar só os termos referentes...
                        tamTermo1=int(fim1)-int(inicio1)
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade=termos_entidade[:tamTermo1]
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        #print('aaaaaaaaaaaaaaaaa')
                        dicEntidades[(int(inicio1), int(fim1))]=[tipo_entidade_string, replaceWhiteSpaces(termos_entidade)]
                        #print("(int(inicio1), int(fim1)):", (int(inicio1), int(fim1)))
                        #print("tipo_entidade_string, replaceWhiteSpaces(termos_entidade):", tipo_entidade_string, replaceWhiteSpaces(termos_entidade))
                        
                        entidade2=entidade_temp[1]
                        inicio2, fim2 = entidade2.split()[0:2]
                        termos_entidade = entidade[2].replace('\n','')
                        termos_entidade=termos_entidade[tamTermo1:len(termos_entidade)]
                        termos_entidade = termos_entidade.strip().replace('/',' / ').replace(')',' ) ').replace('(',' ( ').replace(']',' ] ').replace('[',' [ ').replace(',',' , ').replace('.',' . ').replace(';',' ; ').replace('-',' - ').replace('+',' + ').replace("'"," ' ").replace(" = "," = ").replace("#"," # ").replace(" $ "," $ ").replace(" ! "," ! ").replace("?"," ? ").replace("%"," % ").replace(":"," : ").replace(">"," > ").replace("<"," < ")
                        dicEntidades[(int(inicio2), int(fim2))]=[tipo_entidade_string, replaceWhiteSpaces(termos_entidade)]
                        #print("(int(inicio2), int(fim2)):", (int(inicio2), int(fim2)))
                        #print("tipo_entidade_string, replaceWhiteSpaces(termos_entidade):", tipo_entidade_string, replaceWhiteSpaces(termos_entidade))
                        print('--dicEntidades:--', dicEntidades)

                #print('frasesTokens:', frasesTokens)
                indicesDic = sorted(dicEntidades.keys(), key = lambda item: item[0])
                listaIndicesJaUsados = []
                #list_students.sort(key = lambda x: x[1])   #index 1 means second element
                print(indicesDic)
                #print('dicEntidades:', dicEntidades)
                
                for key, value in frasesTokens.items():
                    print('key:', key)
                    print('value:', value)
                    for i in indicesDic:
                        tipo_entidade,termos_entidade = dicEntidades[i]
                        ##key (i) = indices old a comparar
                        #print('i:', i)
                        for token in value:
                            #print('token:', token)
                            if i[0]==token[2]:
                                novo_inicio, novo_fim = [token[1],token[1]+len(termos_entidade.split())]
                                novos_indices=[]
                                for k in range(novo_inicio,novo_fim):
                                    novos_indices.append(k)
                                listaEntidades.append([termos_entidade, novos_indices, tipo_entidade])
                                print("[termos_entidade, novos_indices, tipo_entidade]:", [termos_entidade, novos_indices, tipo_entidade])
                            else:
                                #print('else, i[0]:',i[0])
                                pass
                        
                    if len(value)>0:
                        #print('incluindo:', key)
                        dic_sentences[num]=[value, listaEntidades]
                        listaEntidades=[]
                        num=num+1 

        #print(num)
        #if num>318:
        #    break

        if num>5:
            break
    print('numMaxTokensPorFrase:', numMaxTokensPorFrase)
    return dic_sentences, frasesComDescontinuas
            
'''

'\ndef getDicSentences(pastaCorpus):\n    #devePrintar=True\n    devePrintar=False\n    dic_sentences = {}\n    numMaxTokensPorFrase=0 # numero tokens da maior frase\n    num=0\n    listaEntidades=[]\n    frasesComDescontinuas=[]\n    for filename in os.listdir(pastaCorpus):\n        f = os.path.join(pastaCorpus, filename)\n        if os.path.isfile(f):\n            fileNameSemExtensao=os.path.splitext(filename)[0]\n            if devePrintar:\n                print(\'\n\n--fileName (sem extensao):--\', fileNameSemExtensao)\n                pass\n            extension = os.path.splitext(filename)[1][1:]\n            if extension==\'ann\':\n                # tokens da frase\n                fileTxt = open(os.path.join(pastaCorpus, fileNameSemExtensao)+\'.txt\', "r", encoding=\'utf-8\')\n                linha=fileTxt.readlines()\n                fileTxt.close()\n                if devePrintar:\n                    print(\'linha:\', linha)\n                    pass\n                frases

In [10]:
dic_sentencesTest, frasesComDescontinuasTest = getDicSentences(pastasCorpus[0])


descontinua, file: 9410_goldstandard_test.ann, num: 0
--dicEntidades:-- {(106, 108): ['Problema', 'FA'], (117, 128): ['Tratamento', 'marevan 5mg'], (142, 154): ['Problema', 'Comorbidades'], (156, 158): ['Problema', 'DM'], (180, 196): ['Tratamento', 'metformina 850mg'], (207, 215): ['Tratamento', 'acarbose'], (227, 241): ['Tratamento', 'glicazida 60mg'], (253, 261): ['Tratamento', 'insulina'], (277, 280): ['Problema', 'HAS'], (302, 316): ['Tratamento', 'losartana 50mg'], (324, 332): ['Tratamento', 'digoxina'], (346, 359): ['Tratamento', 'carvedilol 25'], (367, 371): ['Tratamento', 'HCTZ'], (373, 377): ['Problema', 'DSLP'], (388, 400): ['Tratamento', 'sinvastatina'], (402, 409): ['Tratamento', 'marevan'], (472, 480): ['Problema', 'dispnéia'], (482, 485): ['Problema', 'DPN'], (488, 497): ['Problema', 'ortopneia'], (499, 510): ['Problema', 'palpitações'], (513, 520): ['Problema', 'sincope'], (551, 558): ['Problema', 'afebril'], (560, 562): ['Teste', 'fc'], (567, 570): ['Teste', 'so2'], (57

In [11]:
dic_sentencesTest[10]

[[['MMII', 0, 695],
  ['sem', 1, 700],
  ['edema', 2, 704],
  [',', 3, 709],
  ['panturrilhas', 4, 711],
  ['livres', 5, 724],
  ['.', 6, 730]],
 [['MMII', [0], 'Anatomia'],
  ['edema', [2], 'Problema'],
  ['panturrilhas', [4], 'Anatomia']]]

In [15]:
dic_sentencesTest[1]

[[['Em', 0, 59],
  ['acompanhamento', 1, 62],
  ['no', 2, 77],
  ['ambualtorio', 3, 80],
  ['há', 4, 92],
  ['5', 5, 95],
  ['anos', 6, 97],
  ['por', 7, 102],
  ['FA', 8, 106],
  [',', 9, 108],
  ['uso', 10, 110],
  ['de', 11, 114],
  ['marevan', 12, 117],
  ['5mg', 13, 125],
  ['1', 14, 129],
  ['x', 15, 131],
  ['ao', 16, 133],
  ['dia', 17, 136],
  ['.', 18, 139]],
 [['FA', [8], 'Problema'], ['marevan 5mg', [12, 13], 'Tratamento']]]

In [18]:
dic_sentencesTrain, frasesComDescontinuasTrain = getDicSentences(pastasCorpus[1])


[termos_entidade, novos_indices, tipo_entidade]: ['Dispneia importante aos esforços', [0, 1, 2, 3], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['dor tipo peso no peito no esforço', [5, 6, 7, 8, 9, 10, 11], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['peito', [9], 'Anatomia']
[termos_entidade, novos_indices, tipo_entidade]: ['Obeso', [0], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['has', [2], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['icc', [4], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['cintilografia miocardica', [2, 3], 'Teste']
[termos_entidade, novos_indices, tipo_entidade]: ['miocardica', [3], 'Anatomia']
[termos_entidade, novos_indices, tipo_entidade]: ['avaliar', [5], 'Teste']
[termos_entidade, novos_indices, tipo_entidade]: ['angina', [6], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['Plastia Mitral', [0, 1], 'Tratamento']
[termos_entidade, novos_indices, tipo_entidade

[termos_entidade, novos_indices, tipo_entidade]: ['DAC', [0], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['LESAO REMANESCENTE MODERADA', [3, 4, 5], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['TBG', [2], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['QX CARDILOGICAS', [1, 2], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['CARDILOGICAS', [2], 'Anatomia']
[termos_entidade, novos_indices, tipo_entidade]: ['MUIALGIA', [1], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['DESCONFORTO DE MMII', [3, 4, 5], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['MMII', [5], 'Anatomia']
[termos_entidade, novos_indices, tipo_entidade]: ['AE', [9], 'Anatomia']
[termos_entidade, novos_indices, tipo_entidade]: ['INSUF MI DISCRETA', [11, 12, 13], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['MI', [12], 'Anatomia']
[termos_entidade, novos_indices, tipo_entidade]: ['ISQUEMIA', [23], 'Problema']
[t

[termos_entidade, novos_indices, tipo_entidade]: ['SE', [14], 'Anatomia']
[termos_entidade, novos_indices, tipo_entidade]: ['PP', [17], 'Anatomia']
[termos_entidade, novos_indices, tipo_entidade]: ['VE ACINESIA ANTERO - SEPTAL E ANTERIOR MEDIO - APICAL E APICAL', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['VE', [0], 'Anatomia']
[termos_entidade, novos_indices, tipo_entidade]: ['ANTERO - SEPTAL', [2, 3, 4], 'Anatomia']
[termos_entidade, novos_indices, tipo_entidade]: ['ANTERIOR MEDIO - APICAL', [6, 7, 8, 9], 'Anatomia']
[termos_entidade, novos_indices, tipo_entidade]: ['APICAL', [11], 'Anatomia']
[termos_entidade, novos_indices, tipo_entidade]: ['PA', [2], 'Teste']
[termos_entidade, novos_indices, tipo_entidade]: ['FC', [7], 'Teste']
[termos_entidade, novos_indices, tipo_entidade]: ['SAT', [9], 'Teste']
[termos_entidade, novos_indices, tipo_entidade]: ['SRA', [4], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['AB

[termos_entidade, novos_indices, tipo_entidade]: ['ventricular', [10], 'Anatomia']
[termos_entidade, novos_indices, tipo_entidade]: ['sopro carotideo', [1, 2], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['carotideo', [2], 'Anatomia']
[termos_entidade, novos_indices, tipo_entidade]: ['PA', [0], 'Teste']
[termos_entidade, novos_indices, tipo_entidade]: ['FC', [6], 'Teste']
[termos_entidade, novos_indices, tipo_entidade]: ['sopro', [9], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['ruidos adventicios', [5, 6], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['Ext', [0], 'Anatomia']
[termos_entidade, novos_indices, tipo_entidade]: ['edema', [7], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['TVNS', [0], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['estudo eletrofisiológicos', [1, 2], 'Teste']
[termos_entidade, novos_indices, tipo_entidade]: ['atenolol', [6], 'Tratamento']
[termos_entidade, novos_indices, tipo_

[termos_entidade, novos_indices, tipo_entidade]: ['arritmia', [1], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['exames', [1], 'Teste']
[termos_entidade, novos_indices, tipo_entidade]: ['palpitações', [1], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['dores por artrose', [1, 2, 3], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['dipirona', [6], 'Tratamento']
[termos_entidade, novos_indices, tipo_entidade]: ['muita dor', [9, 10], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['cardíacos', [4], 'Anatomia']
[termos_entidade, novos_indices, tipo_entidade]: ['Hipertensa', [0], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['Hipotireoidismo', [0], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['queda de cabelo', [3, 4, 5], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['cabelo', [5], 'Anatomia']
[termos_entidade, novos_indices, tipo_entidade]: ['Menopausa', [0], 'Problema']
[termos_en

[termos_entidade, novos_indices, tipo_entidade]: ['TX CARDIACO', [1, 2], 'Tratamento']
[termos_entidade, novos_indices, tipo_entidade]: ['CARDIACO', [2], 'Anatomia']
[termos_entidade, novos_indices, tipo_entidade]: ['BIOPSIA', [1], 'Teste']
[termos_entidade, novos_indices, tipo_entidade]: ['CICLO', [3], 'Tratamento']
[termos_entidade, novos_indices, tipo_entidade]: ['AZATIOPRINA 50', [8, 9], 'Tratamento']
[termos_entidade, novos_indices, tipo_entidade]: ['omeprazol', [2], 'Tratamento']
[termos_entidade, novos_indices, tipo_entidade]: ['AAS 100', [4, 5], 'Tratamento']
[termos_entidade, novos_indices, tipo_entidade]: ['ATENOLOL 50', [7, 8], 'Tratamento']
[termos_entidade, novos_indices, tipo_entidade]: ['DOSAGEM CICLOSPORINA', [0, 1], 'Teste']
[termos_entidade, novos_indices, tipo_entidade]: ['QUEIXAS', [3], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['PA', [2], 'Teste']
[termos_entidade, novos_indices, tipo_entidade]: ['FC', [7], 'Teste']
[termos_entidade, novos_indice

[termos_entidade, novos_indices, tipo_entidade]: ['exames', [5], 'Teste']
[termos_entidade, novos_indices, tipo_entidade]: ['HAS', [0], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['hipotiroidismo', [2], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['dislipidemia', [4], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['síndrome X', [6, 7], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['enalapril 20 mg', [3, 4, 5], 'Tratamento']
[termos_entidade, novos_indices, tipo_entidade]: ['AAS 100 mg', [10, 11, 12], 'Tratamento']
[termos_entidade, novos_indices, tipo_entidade]: ['carvedilol 6 , 25 mg', [14, 15, 16, 17, 18], 'Tratamento']
[termos_entidade, novos_indices, tipo_entidade]: ['hidroclorotiazida 25 mg', [24, 25, 26], 'Tratamento']
[termos_entidade, novos_indices, tipo_entidade]: ['rosuvastatina 10 mg', [28, 29, 30], 'Tratamento']
[termos_entidade, novos_indices, tipo_entidade]: ['ezetinibe 10 mg', [32, 33, 34], 'Tratamento']
[

[termos_entidade, novos_indices, tipo_entidade]: ['HAS', [0], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['DLP', [2], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['Bronquite', [0], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['Obesidade', [0], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['Tosse', [1], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['captopril', [3], 'Tratamento']
[termos_entidade, novos_indices, tipo_entidade]: ['Losartana 100', [1, 2], 'Tratamento']
[termos_entidade, novos_indices, tipo_entidade]: ['anlodipino 5', [6, 7], 'Tratamento']
[termos_entidade, novos_indices, tipo_entidade]: ['HCTZ 25', [11, 12], 'Tratamento']
[termos_entidade, novos_indices, tipo_entidade]: ['Alenia', [14], 'Tratamento']
[termos_entidade, novos_indices, tipo_entidade]: ['dor em MSE e irradiação para região precordial', [2, 3, 4, 5, 6, 7, 8, 9], 'Problema']
[termos_entidade, novos_indices, tipo_entidade]: ['MSE

In [8]:
print('--Treinamento:--', pastasCorpus[1])
dic_sentencesTrain, frasesComDescontinuasTrain = getDicSentences(pastasCorpus[1])
print('len(sentences):', len(dic_sentencesTrain))
print('len(Descontinuas):',len(frasesComDescontinuasTrain))
print('len(frasesComDescontinuas):',len(set(frasesComDescontinuasTrain)))


print('--Teste:--', pastasCorpus[0])
dic_sentencesTest, frasesComDescontinuasTest = getDicSentences(pastasCorpus[0])
print('len(sentences):', len(dic_sentencesTest))
print('len(Descontinuas):',len(frasesComDescontinuasTest))
print('len(frasesComDescontinuas):',len(set(frasesComDescontinuasTest)))


--Treinamento:-- corpus\train
numMaxTokensPorFrase: 192
len(sentences): 1736
len(Descontinuas): 91
len(frasesComDescontinuas): 51
--Teste:-- corpus\test
numMaxTokensPorFrase: 146
len(sentences): 506
len(Descontinuas): 44
len(frasesComDescontinuas): 17


In [9]:
dic_sentencesTest[9]

[[['Abd', 0, 644],
  ['globoso', 1, 648],
  [',', 2, 655],
  ['flacido', 3, 657],
  [',', 4, 664],
  ['indolor', 5, 666],
  ['a', 6, 674],
  ['palpacao', 7, 676],
  [',', 8, 684],
  ['sem', 9, 686],
  ['VCM', 10, 690],
  ['.', 11, 693]],
 [['Abd', [0], 'Anatomia'], ['VCM', [10], 'Problema']]]

In [10]:
dic_sentencesTrain[861]

[[['CONTRAÇÃO', 0, 507],
  ['SEGMENTAR', 1, 517],
  ['DO', 2, 527],
  ['VE', 3, 530],
  ['ALTERADA', 4, 533],
  ['.', 5, 541]],
 [['CONTRAÇÃO SEGMENTAR DO VE ALTERADA', [0, 1, 2, 3, 4], 'Problema'],
  ['VE', [3], 'Anatomia']]]

In [11]:
frasesComDescontinuasTrain[:10]

[3, 15, 30, 30, 30, 30, 55, 55, 65, 65]

In [12]:
# frases com entidades descontinuas
print(len(set(frasesComDescontinuasTrain)))

51


In [9]:
frasesComDescontinuasTest

[0,
 42,
 42,
 42,
 49,
 67,
 67,
 67,
 67,
 67,
 91,
 131,
 147,
 197,
 197,
 197,
 197,
 197,
 197,
 197,
 197,
 197,
 197,
 197,
 227,
 241,
 241,
 241,
 241,
 241,
 241,
 241,
 263,
 291,
 291,
 291,
 316,
 329,
 367,
 367,
 367,
 367,
 406,
 468]

In [13]:
print(dic_sentencesTest[2])

[[['Comorbidades', 0, 142], [':', 1, 154], ['DM', 2, 156], ['há', 3, 159], ['10', 4, 162], ['anos', 5, 165], ['em', 6, 170], ['uso', 7, 173], ['de', 8, 177], ['metformina', 9, 180], ['850mg', 10, 191], ['3', 11, 197], ['cp', 12, 199], ['/', 13, 201], ['dia', 14, 202], [',', 15, 205], ['acarbose', 16, 207], ['1', 17, 216], ['cp', 18, 218], ['/', 19, 220], ['dia', 20, 221], ['e', 21, 225], ['glicazida', 22, 227], ['60mg', 23, 237], ['2', 24, 242], ['cp', 25, 244], ['/', 26, 246], ['dia', 27, 247], ['e', 28, 251], ['insulina', 29, 253], ['(', 30, 262], ['24', 31, 263], ['-', 32, 266], ['0', 33, 268], ['-', 34, 270], ['24', 35, 272], [')', 36, 274], ['.', 37, 275]], [['Comorbidades', [0], 'Problema'], ['DM', [2], 'Problema'], ['metformina 850mg', [9, 10], 'Tratamento'], ['acarbose', [16], 'Tratamento'], ['glicazida 60mg', [22, 23], 'Tratamento'], ['insulina', [29], 'Tratamento']]]


In [14]:
save_obj('dic_sentencesTrainDev',dic_sentencesTrain)
porc=0.76
tamanhoTotal = len(dic_sentencesTrain)
tamanhoTrain = int(tamanhoTotal*porc)
print('tamanhoTrain', tamanhoTrain)
tamanhoDev = tamanhoTotal - tamanhoTrain
print('tamanhoDev', tamanhoDev)
dic_sentencesDev_temp = {k: dic_sentencesTrain[k] for k in list(dic_sentencesTrain)[tamanhoTrain:-1]}
dic_sentencesTrain = {k: dic_sentencesTrain[k] for k in list(dic_sentencesTrain)[:tamanhoTrain]}
num=0
dic_sentencesDev = {}
for key, value in dic_sentencesDev_temp.items():
    dic_sentencesDev[num] = value
    num=num+1

print('len(dic_sentencesTrain):', len(dic_sentencesTrain))
print('len(dic_sentencesDev):', len(dic_sentencesDev))
print(dic_sentencesTrain[tamanhoTrain-1])
print(dic_sentencesDev[0])

tamanhoTrain 1319
tamanhoDev 417
len(dic_sentencesTrain): 1319
len(dic_sentencesDev): 416
[[['Paciente', 0, 151], ['relata', 1, 160], ['apenas', 2, 167], ['um', 3, 174], ['episodio', 4, 177], ['no', 5, 186], ['momento', 6, 189], ['de', 7, 197], ['gripe', 8, 200], ['.', 9, 205]], [['gripe', [8], 'Problema']]]
[[['HAS', 0, 207], [',', 1, 210], ['ICC', 2, 212], [',', 3, 215], ['nega', 4, 217], ['DM', 5, 222], ['.', 6, 224]], [['HAS', [0], 'Problema'], ['ICC', [2], 'Problema'], ['DM', [5], 'Problema']]]


In [15]:
dic_sentencesDev[0]

[[['HAS', 0, 207],
  [',', 1, 210],
  ['ICC', 2, 212],
  [',', 3, 215],
  ['nega', 4, 217],
  ['DM', 5, 222],
  ['.', 6, 224]],
 [['HAS', [0], 'Problema'], ['ICC', [2], 'Problema'], ['DM', [5], 'Problema']]]

In [16]:
dic_sentencesDev[1]

[[['carvedilol', 0, 226],
  ['1', 1, 237],
  ['cp', 2, 239],
  ['12', 3, 242],
  ['/', 4, 244],
  ['12', 5, 245],
  [',', 6, 247],
  ['furosemida', 7, 249],
  ['20mg', 8, 260],
  ['2', 9, 265],
  ['cp', 10, 267],
  ['de', 11, 270],
  ['12', 12, 273],
  ['/', 13, 275],
  ['12', 14, 276],
  [',', 15, 278],
  ['sinvastatina', 16, 280],
  ['1cp', 17, 293],
  ['a', 18, 297],
  ['noite', 19, 299],
  [',', 20, 304],
  ['AAS', 21, 306],
  ['100mg', 22, 310],
  ['apos', 23, 316],
  ['almoço', 24, 321],
  ['e', 25, 328],
  ['Omeprazol', 26, 330],
  ['20mg', 27, 340],
  ['1', 28, 345],
  ['xdia', 29, 347],
  ['.', 30, 351]],
 [['carvedilol', [0], 'Tratamento'],
  ['furosemida 20mg', [7, 8], 'Tratamento'],
  ['sinvastatina', [16], 'Tratamento'],
  ['AAS 100mg', [21, 22], 'Tratamento'],
  ['Omeprazol 20mg', [26, 27], 'Tratamento']]]

In [17]:
save_obj('dic_sentencesTrain',dic_sentencesTrain)
save_obj('dic_sentencesDev',dic_sentencesDev)
save_obj('dic_sentencesTest',dic_sentencesTest)

In [18]:
# gerar arquivo treinamento
path='data-ner'
f_train = open(path+r'\nested_train.conll', 'w', encoding='utf-8')
print('Gravando em ', path)
num_entidade_total=0
num_entidade_train=0
num_entidade_dev=0
num_entidade_test=0


for i in range(len(dic_sentencesTrain)):
    tokens = dic_sentencesTrain[i][0]
    ents = dic_sentencesTrain[i][1]
    indiceEnts=[]
    for token in tokens:
        #print('token:', token)
        indiceToken = token[1]
        tag='O'
        for ent in ents:
            if indiceToken in ent[1]:
                tag = ent[2]
                break
        tokenGravar = token[0].replace(' ','')
        tokenGravar = tokenGravar.strip()
        f_train.write(tokenGravar+' '+tag+'\n')
        num_entidade_train=num_entidade_train+1
    f_train.write('\n')
        
f_train.close()

f_dev = open(path+r'\nested_dev.conll', 'w', encoding='utf-8')
for i in range(len(dic_sentencesDev)):
    tokens = dic_sentencesDev[i][0]
    ents = dic_sentencesDev[i][1]
    indiceEnts=[]
    for token in tokens:
        #print('token:', token)
        indiceToken = token[1]
        tag='O'
        for ent in ents:
            if indiceToken in ent[1]:
                tag = ent[2]
                break
        tokenGravar = token[0].replace(' ','')
        tokenGravar = tokenGravar.strip()
        f_dev.write(tokenGravar+' '+tag+'\n')
        num_entidade_dev=num_entidade_dev+1
    f_dev.write('\n')
f_dev.close()


f_test = open(path+r'\nested_test.conll', 'w', encoding='utf-8')
for i in range(len(dic_sentencesTest)):
    tokens = dic_sentencesTest[i][0]
    ents = dic_sentencesTest[i][1]
    indiceEnts=[]
    for token in tokens:
        #print('token:', token)
        indiceToken = token[1]
        tag='O'
        for ent in ents:
            if indiceToken in ent[1]:
                tag = ent[2]
                break
        tokenGravar = token[0].replace(' ','')
        tokenGravar = tokenGravar.strip()
        f_test.write(tokenGravar+' '+tag+'\n')
        num_entidade_test=num_entidade_test+1
    f_test.write('\n')
f_test.close()

print('num_entidade_train', num_entidade_train)
print('num_entidade_dev', num_entidade_dev)
print('num_entidade_test:', num_entidade_test)
num_entidade_total=num_entidade_train+num_entidade_dev+num_entidade_test
print('num_entidade_total:', num_entidade_total)

save_obj('dic_sentencesTrain',dic_sentencesTrain)
save_obj('dic_sentencesDev',dic_sentencesDev)
save_obj('dic_sentencesTest',dic_sentencesTest)


Gravando em  data-ner
num_entidade_train 13801
num_entidade_dev 4009
num_entidade_test: 5453
num_entidade_total: 23263


In [19]:
dic_sentencesTrain[0]

[[['Dispneia', 0, 43],
  ['importante', 1, 52],
  ['aos', 2, 63],
  ['esforços', 3, 67],
  ['+', 4, 76],
  ['dor', 5, 78],
  ['tipo', 6, 82],
  ['peso', 7, 87],
  ['no', 8, 92],
  ['peito', 9, 95],
  ['no', 10, 101],
  ['esforço', 11, 104],
  ['.', 12, 111]],
 [['Dispneia importante aos esforços', [0, 1, 2, 3], 'Problema'],
  ['dor tipo peso no peito no esforço', [5, 6, 7, 8, 9, 10, 11], 'Problema'],
  ['peito', [9], 'Anatomia']]]

### Gerar modelos binarios para baseline

In [20]:
def gravarArquivosBinarios(entidade, dic_sentences, tipo):
    # gerar arquivo treinamento
    f_entidade = open(r'binarios/nested_'+tipo+'_'+entidade+'.conll', 'w', encoding='utf-8')

    num_entidade_total=0
    num_entidade=0

    # TODO - refazer.. qdo vem entidade isolada, nao está gravando...
    print('\nGravando arquivo de {} para entidade {}'.format(tipo, entidade))

    for i in range(len(dic_sentences)):
        tokens = dic_sentences[i][0]
        ents = dic_sentences[i][1]
        indiceEnts=[]
        for token in tokens:
            #print('token:', token)
            indiceToken = token[1]
            tag='O'
            for ent in ents:
                if indiceToken in ent[1] and ent[2]==entidade:
                    tag = ent[2]
                    num_entidade=num_entidade+1
                    break
            if tag != entidade:
                tag='O'
            tokenGravar = token[0].replace(' ','')
            tokenGravar = tokenGravar.strip()
            f_entidade.write(tokenGravar+' '+tag+'\n')
            num_entidade_total=num_entidade_total+1
        f_entidade.write('\n')

    f_entidade.close()

    print('num_entidade:', num_entidade)
    print('num_entidade_total:', num_entidade_total)



In [21]:
entidades = ['Problema','Teste','Tratamento','Anatomia']
#entidades = ['Anatomia']
for entidade in entidades:
    gravarArquivosBinarios(entidade, dic_sentencesTrain, 'train')
    gravarArquivosBinarios(entidade, dic_sentencesDev, 'dev')
    gravarArquivosBinarios(entidade, dic_sentencesTest, 'test')



Gravando arquivo de train para entidade Problema
num_entidade: 2297
num_entidade_total: 13801

Gravando arquivo de dev para entidade Problema
num_entidade: 538
num_entidade_total: 4009

Gravando arquivo de test para entidade Problema
num_entidade: 795
num_entidade_total: 5453

Gravando arquivo de train para entidade Teste
num_entidade: 754
num_entidade_total: 13801

Gravando arquivo de dev para entidade Teste
num_entidade: 172
num_entidade_total: 4009

Gravando arquivo de test para entidade Teste
num_entidade: 308
num_entidade_total: 5453

Gravando arquivo de train para entidade Tratamento
num_entidade: 1191
num_entidade_total: 13801

Gravando arquivo de dev para entidade Tratamento
num_entidade: 377
num_entidade_total: 4009

Gravando arquivo de test para entidade Tratamento
num_entidade: 450
num_entidade_total: 5453

Gravando arquivo de train para entidade Anatomia
num_entidade: 582
num_entidade_total: 13801

Gravando arquivo de dev para entidade Anatomia
num_entidade: 139
num_entida

### Gravar arquivo para multilabel

In [22]:
def gravarArquivosMultilabel(dic_sentences, tipo):
    # gerar arquivo treinamento
    f_entidade = open(r'multilabel/nested_'+tipo+'.conll', 'w', encoding='utf-8')

    num_entidade_total=0

    print('\nGravando arquivo multilabel de {}'.format(tipo))

    entidades = ['Problema','Teste','Tratamento','Anatomia']
    for i in range(len(dic_sentences)):
        tokens = dic_sentences[i][0]
        ents = dic_sentences[i][1]
        #print('tokens:', tokens)
        #print('ents:', ents)
        for token in tokens:
            listaTokensLabels = list()
            #print('token:', token)
            indiceToken = token[1]
            #print('indiceToken:', indiceToken)
            labels = list()
            for ent in ents:
                #print('ent:', ent)
                #print('indiceToken:', indiceToken)
                if indiceToken in ent[1]:
                    for entidade in entidades:
                        tag = ent[2]
                        if tag==entidade:
                            labels.append(tag)
                            num_entidade_total=num_entidade_total+1
                        else:
                            labels.append('O')
            if len(labels)==0:
                labels = ['O','O','O','O']
            tokenGravar = token[0].replace(' ','')
            tokenGravar = tokenGravar.strip()
            f_entidade.write(tokenGravar+'\t'+'\t'.join(labels)+'\n')
            #print(tokenGravar+'\t'+'\t'.join(labels))

        
        f_entidade.write('\n')

    f_entidade.close()

    print('num_entidade_total:', num_entidade_total)



In [23]:
gravarArquivosMultilabel(dic_sentencesTrain, 'train')
gravarArquivosMultilabel(dic_sentencesDev, 'dev')
gravarArquivosMultilabel(dic_sentencesTest, 'test')



Gravando arquivo multilabel de train
num_entidade_total: 4830

Gravando arquivo multilabel de dev
num_entidade_total: 1226

Gravando arquivo multilabel de test
num_entidade_total: 1803


## parte 2- Gerar arquivo treinamento para SpanClassification


In [18]:
def areConsecutive(arr):
    # Sort the array
    arr.sort()
    n = len(arr)
    # checking the adjacent elements
    for i in range (1,n):
        if(arr[i]!=arr[i-1]+1):
            return False;
             
    return True;

In [56]:
dic_sentencesTest = load_obj('dic_sentencesTest')

In [57]:
def getCombinacaoEntidades(dic_predictions, filtro_postagger, dicPosTagger, taxaDownsampling):
    num=0
    erro_corpus=0
    num_frases_sem_entidade=0
    lista_erro_corpus=list()
    combinacaoEntidadesAll = list()
    combinacaoEntidades = list()
    pulando_termos_postagger = list()
    if filtro_postagger:
        print('Com filtro-postagger')
    else:
        print('Sem filtro-postagger')
    if taxaDownsampling>0:
        print('Com taxa de Downsampling de ', taxaDownsampling)
    else:
        print('Sem taxa de Downsampling')
    for key, value in dic_predictions.items():
        num=num+1
        combinacaoEntidades = list()
        tokens=value[0].copy()
        so_tokens = [t[0] for t in tokens]
        entidades=value[1].copy()
        num_positivas=0
        for entidade in entidades:
            erros_entidade = list()
            texto_entidade=entidade[0].strip()
            indices = entidade[1]
            tipo_entidade = entidade[2]
            frase = so_tokens.copy()
            inicio=indices[0]
            fim=indices[-1]
            frase.insert(inicio, '<e1>')
            frase.insert(fim+2, '</e1>')
            if texto_entidade=='-' or texto_entidade=='=' or texto_entidade=='+' or texto_entidade==':' or texto_entidade==',' or texto_entidade=="'" or texto_entidade=='"' or texto_entidade=='.' or texto_entidade==';' or texto_entidade=='/' or texto_entidade=='(' or texto_entidade==')' or texto_entidade=='[' or texto_entidade==']':
                pass
            texto_entidade_comparar=texto_entidade.replace('/','').replace(')','').replace('(','').replace(']','').replace('[','').replace(',','').replace('.','').replace(';','').replace('-','').replace('+','').replace("'",'')
            texto_entidade_comparar = replaceWhiteSpaces(texto_entidade_comparar)
            texto_frase_comparar = ' '.join(frase[inicio+1:fim+2]).strip().replace('/','').replace(')','').replace('(','').replace(']','').replace('[','').replace(',','').replace('.','').replace(';','').replace('-','').replace('+','').replace("'",'')
            texto_frase_comparar = replaceWhiteSpaces(texto_frase_comparar)
            texto_entidade_comparar = texto_entidade_comparar.lower()
            texto_frase_comparar = texto_frase_comparar.lower()
            if (texto_entidade_comparar == texto_frase_comparar):
                num_positivas=num_positivas+1
                combinacaoEntidades.append([' '.join(frase).strip(), tipo_entidade]) # apendando entidades reais
            else:
                print('erro, key:', key)
                erro_corpus=erro_corpus+1
                erros_entidade.append(indices)
                lista_erro_corpus.append([' '.join(frase).strip(), tipo_entidade, ' '.join(so_tokens), entidade])

        for entidade in entidades:
                indices = entidade[1]
                #print('indices:', indices)
                if indices in erros_entidade:
                    continue
                inicio=indices[0]
                fim=indices[-1]
                # agora, fazer a combinacao entre eles.. todas a seguir serão do tipo 'O'           
                for indice in indices:
                    for i in range(indice, fim+1):
                        # ver se nao tem antes
                        frase = so_tokens.copy()
                        #termo = frase[indice:i+2]
                        termo = frase[indice:i+1]
                        #print('--termo--:', termo)
                        frase.insert(indice, '<e1>')
                        frase.insert(i+2, '</e1>')
                        frase_string=' '.join(frase).strip()
                        #print('frase_string:', frase_string)
                        devePular = 0
                        if '. </e1>' in frase_string or ', </e1>' in frase_string  or '; </e1>' in frase_string or '- </e1>' in frase_string  or ': </e1>' in frase_string  or '= </e1>' in frase_string  or '/ </e1>' in frase_string  or '( </e1>' in frase_string  or ') </e1>' in frase_string  or '[ </e1>' in frase_string  or '] </e1>' in frase_string  or ': </e1>' in frase_string or 'and </e1>' in frase_string or 'or </e1>' in frase_string:
                            devePular=1
                        if '<e1> .' in frase_string or '<e1> ,' in frase_string  or '<e1> ;' in frase_string or '<e1> -' in frase_string  or '<e1> :' in frase_string  or '<e1> =' in frase_string  or '<e1> /' in frase_string  or '<e1> (' in frase_string  or '<e1> )' in frase_string  or '<e1> [' in frase_string  or '<e1> ]' in frase_string  or '<e1> :' in frase_string  or '<e1> and' in frase_string  or '<e1> or' in frase_string:
                            devePular=1
                        if re.search("<e1> [0-9]* </e1>", frase_string):
                            devePular=1
                        if filtro_postagger==True:
                            pos_tagger_termo = tipoPostaggerTokens(termo, dicPosTagger)
                            if pos_tagger_termo not in lista_postaggers_entidades:
                                pulando_termos_postagger.append([termo, pos_tagger_termo])
                                devePular=1
                
                        tem_frase = 0
                        for frase in combinacaoEntidades:
                            if frase[0] == frase_string:
                                tem_frase=''
                                break
                        if tem_frase==0 and devePular==0:
                            combinacaoEntidades.append([frase_string, 'O'])
        # shuffle no combinacaoEntidades
        # taxaDownsampling, ex 2 para o dobro, 1 para mesma quantidade
        if len(combinacaoEntidades)>0:
            if taxaDownsampling>0:
                combinacaoEntidades = combinacaoEntidades[:(num_positivas*taxaDownsampling)+num_positivas]
            random.shuffle(combinacaoEntidades)
            combinacaoEntidadesAll.append([' '.join(so_tokens).strip(), combinacaoEntidades])
        else:
            num_frases_sem_entidade = num_frases_sem_entidade+1
            combinacaoEntidadesAll.append([])
        combinacaoEntidades = list()
        if (num % 1000) ==0:
            print('key:', key)

    print('erro_corpus:', erro_corpus)
    print('num_frases_sem_entidade:', num_frases_sem_entidade)
    print('len(combinacaoEntidadesAll:)',len(combinacaoEntidadesAll))
    print('len(pulando_termos_postagger):', len(pulando_termos_postagger))
    
    return combinacaoEntidadesAll, pulando_termos_postagger

In [27]:
# labels sempre tem que começar com zero, senao da erro no treinamento
# RuntimeError: CUDA error: device-side assert triggered
def getCombinacaoEntidadesSoPositivos(dic_predictions):
    num=0
    erro_corpus=0
    num_frases_sem_entidade=0
    lista_erro_corpus=list()
    combinacaoEntidadesAll = list()
    combinacaoEntidades = list()
    print('Só positivos')
    for key, value in dic_predictions.items():
        num=num+1
        combinacaoEntidades = list()
        tokens=value[0].copy()
        so_tokens = [t[0] for t in tokens]
        entidades=value[1].copy()
        num_positivas=0
        for entidade in entidades:
            erros_entidade = list()
            texto_entidade=entidade[0].strip()
            indices = entidade[1]
            tipo_entidade = entidade[2]
            #print('tipo_entidade:', tipo_entidade)
            frase = so_tokens.copy()
            inicio=indices[0]
            fim=indices[-1]
            frase.insert(inicio, '<e1>')
            frase.insert(fim+2, '</e1>')
            if texto_entidade=='-' or texto_entidade=='=' or texto_entidade=='+' or texto_entidade==':' or texto_entidade==',' or texto_entidade=="'" or texto_entidade=='"' or texto_entidade=='.' or texto_entidade==';' or texto_entidade=='/' or texto_entidade=='(' or texto_entidade==')' or texto_entidade=='[' or texto_entidade==']':
                pass
            texto_entidade_comparar=texto_entidade.replace('/','').replace(')','').replace('(','').replace(']','').replace('[','').replace(',','').replace('.','').replace(';','').replace('-','').replace('+','').replace("'",'')
            texto_entidade_comparar = replaceWhiteSpaces(texto_entidade_comparar)
            texto_frase_comparar = ' '.join(frase[inicio+1:fim+2]).strip().replace('/','').replace(')','').replace('(','').replace(']','').replace('[','').replace(',','').replace('.','').replace(';','').replace('-','').replace('+','').replace("'",'')
            texto_frase_comparar = replaceWhiteSpaces(texto_frase_comparar)
            texto_entidade_comparar = texto_entidade_comparar.lower()
            texto_frase_comparar = texto_frase_comparar.lower()
            if (texto_entidade_comparar == texto_frase_comparar):
                num_positivas=num_positivas+1
                combinacaoEntidades.append([' '.join(frase).strip(), tipo_entidade]) # apendando entidades reais
            else:
                print('erro, key:', key)
                erro_corpus=erro_corpus+1
                erros_entidade.append(indices)
                lista_erro_corpus.append([' '.join(frase).strip(), tipo_entidade, ' '.join(so_tokens), entidade])

        # shuffle no combinacaoEntidades
        if len(combinacaoEntidades)>0:
            random.shuffle(combinacaoEntidades)
            combinacaoEntidadesAll.append([' '.join(so_tokens).strip(), combinacaoEntidades])
        else:
            num_frases_sem_entidade = num_frases_sem_entidade+1
            combinacaoEntidadesAll.append([])
        combinacaoEntidades = list()
  
    print('erro_corpus:', erro_corpus)
    print('num_frases_sem_entidade:', num_frases_sem_entidade)
    print('len(combinacaoEntidadesAll:)',len(combinacaoEntidadesAll))
    
    return combinacaoEntidadesAll

In [28]:
print('--Train--')
combinacaoEntidadesTrainPos= getCombinacaoEntidadesSoPositivos(dic_sentencesTrain)
print('\n--Dev--')
combinacaoEntidadesDevPos = getCombinacaoEntidadesSoPositivos(dic_sentencesDev)
print('\n--Test--')
combinacaoEntidadesTestPos = getCombinacaoEntidadesSoPositivos(dic_sentencesTest)

--Train--
Só positivos
erro_corpus: 0
num_frases_sem_entidade: 237
len(combinacaoEntidadesAll:) 1319

--Dev--
Só positivos
erro_corpus: 0
num_frases_sem_entidade: 89
len(combinacaoEntidadesAll:) 416

--Test--
Só positivos
erro_corpus: 0
num_frases_sem_entidade: 107
len(combinacaoEntidadesAll:) 506


In [60]:
print('--Train--')
combinacaoEntidadesTrain, _ = getCombinacaoEntidades(dic_sentencesTrain, False, '', 0)
print('\n--Dev--')
combinacaoEntidadesDev, _ = getCombinacaoEntidades(dic_sentencesDev, False, '', 0)
print('\n--Test--')
combinacaoEntidadesTest, _ = getCombinacaoEntidades(dic_sentencesTest, False, '', 0)

--Train--
Sem filtro-postagger
Sem taxa de Downsampling
key: 999
erro_corpus: 0
num_frases_sem_entidade: 237
len(combinacaoEntidadesAll:) 1319
len(pulando_termos_postagger): 0

--Dev--
Sem filtro-postagger
Sem taxa de Downsampling
erro_corpus: 0
num_frases_sem_entidade: 89
len(combinacaoEntidadesAll:) 416
len(pulando_termos_postagger): 0

--Test--
Sem filtro-postagger
Sem taxa de Downsampling
erro_corpus: 0
num_frases_sem_entidade: 107
len(combinacaoEntidadesAll:) 506
len(pulando_termos_postagger): 0


In [61]:
combinacaoEntidadesTest[14]

['aumento moderado de átrio esquerdo .',
 [['aumento <e1> moderado de átrio </e1> esquerdo .', 'O'],
  ['aumento moderado <e1> de </e1> átrio esquerdo .', 'O'],
  ['<e1> aumento moderado de átrio esquerdo </e1> .', 'Problema'],
  ['<e1> aumento moderado de </e1> átrio esquerdo .', 'O'],
  ['aumento moderado <e1> de átrio esquerdo </e1> .', 'O'],
  ['aumento <e1> moderado de </e1> átrio esquerdo .', 'O'],
  ['<e1> aumento moderado </e1> de átrio esquerdo .', 'O'],
  ['aumento moderado de átrio <e1> esquerdo </e1> .', 'O'],
  ['aumento moderado de <e1> átrio esquerdo </e1> .', 'Anatomia'],
  ['<e1> aumento moderado de átrio </e1> esquerdo .', 'O'],
  ['aumento <e1> moderado </e1> de átrio esquerdo .', 'O'],
  ['<e1> aumento </e1> moderado de átrio esquerdo .', 'O'],
  ['aumento <e1> moderado de átrio esquerdo </e1> .', 'O'],
  ['aumento moderado <e1> de átrio </e1> esquerdo .', 'O'],
  ['aumento moderado de <e1> átrio </e1> esquerdo .', 'O']]]

In [62]:
# o erro é que o ponto faz com que quebre a frase, mas a entidade continua...
dic_sentencesTrain[829]

[[['CONTRAÇÃO', 0, 507],
  ['SEGMENTAR', 1, 517],
  ['DO', 2, 527],
  ['VE', 3, 530],
  ['ALTERADA', 4, 533],
  ['.', 5, 541]],
 [['CONTRAÇÃO SEGMENTAR DO VE ALTERADA', [0, 1, 2, 3, 4], 'Problema'],
  ['VE', [3], 'Anatomia']]]

In [63]:
combinacaoEntidadesTest[4]

['DSLP em uso de sinvastatina , marevan 1 cp / dia seg - sab para no alvo sic .',
 [['DSLP em uso de <e1> sinvastatina </e1> , marevan 1 cp / dia seg - sab para no alvo sic .',
   'Tratamento'],
  ['DSLP em uso de sinvastatina , <e1> marevan </e1> 1 cp / dia seg - sab para no alvo sic .',
   'Tratamento'],
  ['<e1> DSLP </e1> em uso de sinvastatina , marevan 1 cp / dia seg - sab para no alvo sic .',
   'Problema']]]

In [64]:
combinacaoEntidadesTrain[829]

['CONTRAÇÃO SEGMENTAR DO VE ALTERADA .',
 [['CONTRAÇÃO SEGMENTAR <e1> DO </e1> VE ALTERADA .', 'O'],
  ['CONTRAÇÃO SEGMENTAR DO VE <e1> ALTERADA </e1> .', 'O'],
  ['<e1> CONTRAÇÃO </e1> SEGMENTAR DO VE ALTERADA .', 'O'],
  ['CONTRAÇÃO <e1> SEGMENTAR DO VE </e1> ALTERADA .', 'O'],
  ['CONTRAÇÃO SEGMENTAR DO <e1> VE ALTERADA </e1> .', 'O'],
  ['<e1> CONTRAÇÃO SEGMENTAR DO VE ALTERADA </e1> .', 'Problema'],
  ['CONTRAÇÃO <e1> SEGMENTAR DO </e1> VE ALTERADA .', 'O'],
  ['<e1> CONTRAÇÃO SEGMENTAR DO </e1> VE ALTERADA .', 'O'],
  ['<e1> CONTRAÇÃO SEGMENTAR </e1> DO VE ALTERADA .', 'O'],
  ['CONTRAÇÃO SEGMENTAR <e1> DO VE </e1> ALTERADA .', 'O'],
  ['CONTRAÇÃO <e1> SEGMENTAR DO VE ALTERADA </e1> .', 'O'],
  ['CONTRAÇÃO SEGMENTAR DO <e1> VE </e1> ALTERADA .', 'Anatomia'],
  ['CONTRAÇÃO <e1> SEGMENTAR </e1> DO VE ALTERADA .', 'O'],
  ['<e1> CONTRAÇÃO SEGMENTAR DO VE </e1> ALTERADA .', 'O'],
  ['CONTRAÇÃO SEGMENTAR <e1> DO VE ALTERADA </e1> .', 'O']]]

In [65]:
combinacaoEntidadesDev[0]

['HAS , ICC , nega DM .',
 [['HAS , <e1> ICC </e1> , nega DM .', 'Problema'],
  ['<e1> HAS </e1> , ICC , nega DM .', 'Problema'],
  ['HAS , ICC , nega <e1> DM </e1> .', 'Problema']]]

In [21]:
def gravarArquivosTreinamento(path, combinacaoEntidadesTrain, combinacaoEntidadesDev, combinacaoEntidadesTest):

    # já ir gravando arquivos treinamento, test e dev...
    # pra fazer um teste sem descontinuas
    numTotalEntidades=0
    numTotalEntidadesTrain=0
    numTotalEntidadesDev=0
    numTotalEntidadesTest=0

    existeDir = os.path.exists(path)
    if not existeDir:
        os.makedirs(path)
        
    f_train = open(path+r'\span.train', 'w', encoding='utf-8')

    for i, combinacaoEntidades in enumerate(combinacaoEntidadesTrain):
        if len(combinacaoEntidades)>0:
            frase = combinacaoEntidades[0]
            frases_entidade = combinacaoEntidades[1]
            f_train.write(frase+'\n')
            for frase_entidade in frases_entidade:
                f_train.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
                numTotalEntidades=numTotalEntidades+1
                numTotalEntidadesTrain=numTotalEntidadesTrain+1

    f_train.close()

    f_dev = open(path+r'\span.dev', 'w', encoding='utf-8')

    for i, combinacaoEntidades in enumerate(combinacaoEntidadesDev):
        if len(combinacaoEntidades)>0:
            frase = combinacaoEntidades[0]
            frases_entidade = combinacaoEntidades[1]
            f_dev.write(frase+'\n')
            for frase_entidade in frases_entidade:
                f_dev.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
                numTotalEntidades=numTotalEntidades+1
                numTotalEntidadesDev=numTotalEntidadesDev+1

    f_dev.close()

    f_test = open(path+r'\span.test', 'w', encoding='utf-8')
    for i, combinacaoEntidades in enumerate(combinacaoEntidadesTest):
        #print(dicSentences[i])
        if len(combinacaoEntidades)>0:
            frase = combinacaoEntidades[0]
            frases_entidade = combinacaoEntidades[1]
            f_test.write(frase+'\n')
            for frase_entidade in frases_entidade:
                f_test.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
                numTotalEntidades=numTotalEntidades+1
                numTotalEntidadesTest=numTotalEntidadesTest+1

    f_test.close()

    print('numTotalEntidades:', numTotalEntidades)
    print('numTotalEntidadesTrain:', numTotalEntidadesTrain)
    print('numTotalEntidadesDev:', numTotalEntidadesDev)
    print('numTotalEntidadesTest:', numTotalEntidadesTest)


In [67]:
gravarArquivosTreinamento('sem_filtro',combinacaoEntidadesTrain, combinacaoEntidadesDev, combinacaoEntidadesTest)

numTotalEntidades: 14133
numTotalEntidadesTrain: 9079
numTotalEntidadesDev: 2001
numTotalEntidadesTest: 3053


In [68]:
gravarArquivosTreinamento('so_positivos',combinacaoEntidadesTrainPos, combinacaoEntidadesDevPos, combinacaoEntidadesTestPos)

numTotalEntidades: 4204
numTotalEntidadesTrain: 2510
numTotalEntidadesDev: 703
numTotalEntidadesTest: 991


### Gravar arquivo para sentence pairs

In [31]:
def getCombinacaoEntidadesSentence(dic_predictions):
    #labels = {0:'O', 1:'Problema', 2:'Tratamento', 3:'Teste', 4:'Anatomia'}
    labels = {'O':0, 'Problema':1, 'Tratamento':2, 'Teste':3, 'Anatomia':4}
    num=0
    erro_corpus=0
    num_frases_sem_entidade=0
    lista_erro_corpus=list()
    combinacaoEntidades = list()
    print('Sentence Pairs - Só positivos')
    for key, value in dic_predictions.items():
        num=num+1
        tokens=value[0].copy()
        so_tokens = [t[0] for t in tokens]
        entidades=value[1].copy()
        num_positivas=0
        for entidade in entidades:
            erros_entidade = list()
            texto_entidade=entidade[0].strip()
            indices = entidade[1]
            tipo_entidade = entidade[2]
            frase = so_tokens.copy()
            inicio=indices[0]
            fim=indices[-1]
            #entidade_frase=frase[inicio:fim+1] # texto_entidade
            entidade_frase=texto_entidade
            #print('entidade_frase:', entidade_frase)
            #print('frase:', frase)
            #print('texto_entidade:', texto_entidade)
            if texto_entidade=='-' or texto_entidade=='=' or texto_entidade=='+' or texto_entidade==':' or texto_entidade==',' or texto_entidade=="'" or texto_entidade=='"' or texto_entidade=='.' or texto_entidade==';' or texto_entidade=='/' or texto_entidade=='(' or texto_entidade==')' or texto_entidade=='[' or texto_entidade==']':
                pass
            texto_entidade_comparar=texto_entidade.replace('/','').replace(')','').replace('(','').replace(']','').replace('[','').replace(',','').replace('.','').replace(';','').replace('-','').replace('+','').replace("'",'')
            texto_entidade_comparar = replaceWhiteSpaces(texto_entidade_comparar)
            texto_frase_comparar = ' '.join(frase[inicio:fim+1]).strip().replace('/','').replace(')','').replace('(','').replace(']','').replace('[','').replace(',','').replace('.','').replace(';','').replace('-','').replace('+','').replace("'",'')
            texto_frase_comparar = replaceWhiteSpaces(texto_frase_comparar)
            texto_entidade_comparar = texto_entidade_comparar.lower()
            texto_frase_comparar = texto_frase_comparar.lower()
            #print('texto_entidade_comparar:', texto_entidade_comparar)
            #print('texto_frase_comparar:', texto_frase_comparar)
            if (texto_entidade_comparar == texto_frase_comparar):
                num_positivas=num_positivas+1
                combinacaoEntidades.append([entidade_frase, ' '.join(frase).strip(), labels[tipo_entidade]]) # apendando entidades reais
            else:
                print('erro, key:', key)
                erro_corpus=erro_corpus+1
                erros_entidade.append(indices)
                lista_erro_corpus.append([' '.join(frase).strip(), tipo_entidade, ' '.join(so_tokens), entidade])

        # shuffle no combinacaoEntidades
        #if len(combinacaoEntidades)>0:
        #    random.shuffle(combinacaoEntidades)
        #    combinacaoEntidadesAll.append(combinacaoEntidades)
        #else:
        #    num_frases_sem_entidade = num_frases_sem_entidade+1
            #combinacaoEntidadesAll.append([])
        #combinacaoEntidades = list()
        
    random.shuffle(combinacaoEntidades)
  
    print('erro_corpus:', erro_corpus)
    print('num_frases_sem_entidade:', num_frases_sem_entidade)
    print('len(combinacaoEntidades:)',len(combinacaoEntidades))
    
    return combinacaoEntidades

In [32]:
re.search("^[0-9]*mg", '12mg')

<re.Match object; span=(0, 4), match='12mg'>

In [33]:
def getCombinacaoEntidadesSentence(dic_predictions, filtro_postagger, dicPosTagger, taxaDownsampling):
    #labels = {0:'O', 1:'Problema', 2:'Tratamento', 3:'Teste', 4:'Anatomia'}
    labels = {'O':0, 'Problema':1, 'Tratamento':2, 'Teste':3, 'Anatomia':4}
    num=0
    erro_corpus=0
    num_frases_sem_entidade=0
    lista_erro_corpus=list()
    combinacaoEntidadesPos = list()
    combinacaoEntidadesNeg = list()
    combinacaoEntidades = list()
    pulando_termos_postagger = list()
    if filtro_postagger:
        print('Sentence Pairs - Com filtro-postagger')
    else:
        print('Sentence Pairs - Sem filtro-postagger')
    if taxaDownsampling>0:
        print('Sentence Pairs - Com taxa de Downsampling de ', taxaDownsampling)
    else:
        print('Sentence Pairs - Sem taxa de Downsampling')

    for key, value in dic_predictions.items():
        num=num+1
        tokens=value[0].copy()
        so_tokens = [t[0] for t in tokens]
        entidades=value[1].copy()
        num_positivas=0
        for entidade in entidades:
            erros_entidade = list()
            texto_entidade=entidade[0].strip()
            indices = entidade[1]
            tipo_entidade = entidade[2]
            frase = so_tokens.copy()
            inicio=indices[0]
            fim=indices[-1]
            #entidade_frase=frase[inicio:fim+1] # texto_entidade
            entidade_frase=texto_entidade
            #print('entidade_frase:', entidade_frase)
            #print('frase:', frase)
            #print('texto_entidade:', texto_entidade)
            if texto_entidade=='-' or texto_entidade=='=' or texto_entidade=='+' or texto_entidade==':' or texto_entidade==',' or texto_entidade=="'" or texto_entidade=='"' or texto_entidade=='.' or texto_entidade==';' or texto_entidade=='/' or texto_entidade=='(' or texto_entidade==')' or texto_entidade=='[' or texto_entidade==']':
                pass
            texto_entidade_comparar=texto_entidade.replace('/','').replace(')','').replace('(','').replace(']','').replace('[','').replace(',','').replace('.','').replace(';','').replace('-','').replace('+','').replace("'",'')
            texto_entidade_comparar = replaceWhiteSpaces(texto_entidade_comparar)
            texto_frase_comparar = ' '.join(frase[inicio:fim+1]).strip().replace('/','').replace(')','').replace('(','').replace(']','').replace('[','').replace(',','').replace('.','').replace(';','').replace('-','').replace('+','').replace("'",'')
            texto_frase_comparar = replaceWhiteSpaces(texto_frase_comparar)
            texto_entidade_comparar = texto_entidade_comparar.lower()
            texto_frase_comparar = texto_frase_comparar.lower()
            #print('texto_entidade_comparar:', texto_entidade_comparar)
            #print('texto_frase_comparar:', texto_frase_comparar)
            if (texto_entidade_comparar == texto_frase_comparar):
                num_positivas=num_positivas+1
                combinacaoEntidadesPos.append([entidade_frase, ' '.join(frase).strip(), labels[tipo_entidade]]) # apendando entidades reais
            else:
                print('erro, key:', key)
                erro_corpus=erro_corpus+1
                erros_entidade.append(indices)
                lista_erro_corpus.append([' '.join(frase).strip(), tipo_entidade, ' '.join(so_tokens), entidade])
        # agora, os negativos
        for entidade in entidades:
                indices = entidade[1]
                #print('indices:', indices)
                if indices in erros_entidade:
                    continue
                inicio=indices[0]
                fim=indices[-1]
                # agora, fazer a combinacao entre eles.. todas a seguir serão do tipo 'O'           
                for indice in indices:
                    for i in range(indice, fim+1):
                        # ver se nao tem antes
                        frase = so_tokens.copy()
                        termo = frase[indice:i+1]
                        #termo=entidade[0].strip()
                        #print('--termo--:', termo)
                        #frase.insert(indice, '<e1>')
                        #frase.insert(i+2, '</e1>')
                        #frase_string=' '.join(frase).strip()
                        #frase_string=texto_entidade
                        #frase_string=termo[0]
                        frase_string=' '.join(termo).strip()
                        #print('frase_string:', frase_string)
                        devePular = 0
                        if '.' in frase_string[-1:] or ',' in frase_string[-1:]  or ';' in frase_string[-1:] or '-' in frase_string[-1:]  or ':' in frase_string[-1:]  or '=' in frase_string[-1:]  or '/' in frase_string[-1:]  or '(' in frase_string[-1:]  or ')' in frase_string[-1:]  or '[' in frase_string[-1:]  or ']' in frase_string[-1:]  or ':' in frase_string[-1:]:
                            devePular=1
                        if '.' in frase_string[:1] or ',' in frase_string[:1]  or ';' in frase_string[:1] or '-' in frase_string[:1]  or ':' in frase_string[:1]  or '=' in frase_string[:1] or '/' in frase_string[:1]  or '(' in frase_string[:1]  or ')' in frase_string[:1] or '[' in frase_string[:1]  or ']' in frase_string[:1]  or ':' in frase_string[:1]:
                            devePular=1
                        if re.search("^[0-9]*mg", frase_string):
                            devePular=1
                            
                        if filtro_postagger==True:
                            pos_tagger_termo = tipoPostaggerTokens(termo, dicPosTagger)
                            if pos_tagger_termo not in lista_postaggers_entidades:
                                pulando_termos_postagger.append([termo, pos_tagger_termo])
                                devePular=1
                
                        tem_frase = 0
                        for frase_l in combinacaoEntidadesPos:
                            if frase_l[0] == frase_string:
                                tem_frase='1'
                                break
                        if tem_frase==0 and devePular==0:
                        #print('tem_frase:', tem_frase)
                        #if tem_frase==0:
                            #print('aaaaaaaaaaaa, frase_string:', frase_string)
                            combinacaoEntidadesNeg.append([frase_string, ' '.join(frase).strip(), labels['O']])
                        #combinacaoEntidadesNeg.append([frase_string, ' '.join(frase).strip(), labels['O']])
                        
        # shuffle no combinacaoEntidades
        #if len(combinacaoEntidades)>0:
        #    random.shuffle(combinacaoEntidades)
        #    combinacaoEntidadesAll.append(combinacaoEntidades)
        #else:
        #    num_frases_sem_entidade = num_frases_sem_entidade+1
            #combinacaoEntidadesAll.append([])
        #combinacaoEntidades = list()
        
        # shuffle no combinacaoEntidades
        # taxaDownsampling, ex 2 para o dobro, 1 para mesma quantidade
        if len(combinacaoEntidadesPos)>0:
            if taxaDownsampling>0:
                combinacaoEntidadesNeg = combinacaoEntidadesNeg[:(num_positivas*taxaDownsampling)]
            random.shuffle(combinacaoEntidadesNeg)
        else:
            num_frases_sem_entidade = num_frases_sem_entidade+1
        if (num % 1000) ==0:
            print('key:', key)

        #print('combinacaoEntidadesNeg:',combinacaoEntidadesNeg)
        combinacaoEntidades = combinacaoEntidades+combinacaoEntidadesPos+combinacaoEntidadesNeg
        combinacaoEntidadesPos=list()
        combinacaoEntidadesNeg=list()
  
    print('erro_corpus:', erro_corpus)
    print('num_frases_sem_entidade:', num_frases_sem_entidade)
    print('len(combinacaoEntidades:)',len(combinacaoEntidades))
    
    return combinacaoEntidades

#combinacaoEntidadesTestSentence = getCombinacaoEntidadesSentence(dic_sentencesTest, True, dicPosTagger, 1)
#combinacaoEntidadesTrain, pulando_termos_postaggerTrain = getCombinacaoEntidades(dic_sentencesTrain, True, dicPosTagger, 0)
#combinacaoEntidadesTestSentence

In [40]:
def getCombinacaoEntidadesSentencePos(dic_predictions):
    labels = {'Problema':0, 'Tratamento':1, 'Teste':2, 'Anatomia':3}
    num=0
    erro_corpus=0
    num_frases_sem_entidade=0
    lista_erro_corpus=list()
    combinacaoEntidadesPos = list()
    combinacaoEntidades = list()
    pulando_termos_postagger = list()
    print('Sentence Pairs - So positivos')

    for key, value in dic_predictions.items():
        num=num+1
        tokens=value[0].copy()
        so_tokens = [t[0] for t in tokens]
        entidades=value[1].copy()
        num_positivas=0
        for entidade in entidades:
            erros_entidade = list()
            texto_entidade=entidade[0].strip()
            indices = entidade[1]
            tipo_entidade = entidade[2]
            frase = so_tokens.copy()
            inicio=indices[0]
            fim=indices[-1]
            #entidade_frase=frase[inicio:fim+1] # texto_entidade
            entidade_frase=texto_entidade
            #print('entidade_frase:', entidade_frase)
            #print('frase:', frase)
            #print('texto_entidade:', texto_entidade)
            if texto_entidade=='-' or texto_entidade=='=' or texto_entidade=='+' or texto_entidade==':' or texto_entidade==',' or texto_entidade=="'" or texto_entidade=='"' or texto_entidade=='.' or texto_entidade==';' or texto_entidade=='/' or texto_entidade=='(' or texto_entidade==')' or texto_entidade=='[' or texto_entidade==']':
                pass
            texto_entidade_comparar=texto_entidade.replace('/','').replace(')','').replace('(','').replace(']','').replace('[','').replace(',','').replace('.','').replace(';','').replace('-','').replace('+','').replace("'",'')
            texto_entidade_comparar = replaceWhiteSpaces(texto_entidade_comparar)
            texto_frase_comparar = ' '.join(frase[inicio:fim+1]).strip().replace('/','').replace(')','').replace('(','').replace(']','').replace('[','').replace(',','').replace('.','').replace(';','').replace('-','').replace('+','').replace("'",'')
            texto_frase_comparar = replaceWhiteSpaces(texto_frase_comparar)
            texto_entidade_comparar = texto_entidade_comparar.lower()
            texto_frase_comparar = texto_frase_comparar.lower()
            #print('texto_entidade_comparar:', texto_entidade_comparar)
            #print('texto_frase_comparar:', texto_frase_comparar)
            if (texto_entidade_comparar == texto_frase_comparar):
                num_positivas=num_positivas+1
                combinacaoEntidadesPos.append([entidade_frase, ' '.join(frase).strip(), labels[tipo_entidade]]) # apendando entidades reais
            else:
                print('erro, key:', key)
                erro_corpus=erro_corpus+1
                erros_entidade.append(indices)
                lista_erro_corpus.append([' '.join(frase).strip(), tipo_entidade, ' '.join(so_tokens), entidade])

        if len(combinacaoEntidadesPos)>0:
            random.shuffle(combinacaoEntidadesPos)
        else:
            num_frases_sem_entidade = num_frases_sem_entidade+1
        if (num % 1000) ==0:
            print('key:', key)

        #print('combinacaoEntidadesNeg:',combinacaoEntidadesNeg)
        combinacaoEntidades = combinacaoEntidades+combinacaoEntidadesPos
        combinacaoEntidadesPos=list()
  
    print('erro_corpus:', erro_corpus)
    print('num_frases_sem_entidade:', num_frases_sem_entidade)
    print('len(combinacaoEntidades:)',len(combinacaoEntidades))
    
    return combinacaoEntidades

#combinacaoEntidadesTestSentence = getCombinacaoEntidadesSentencePos(dic_sentencesTest)
#combinacaoEntidadesTrain, pulando_termos_postaggerTrain = getCombinacaoEntidades(dic_sentencesTrain, True, dicPosTagger, 0)
#combinacaoEntidadesTestSentence

In [28]:
print('--Train--')
combinacaoEntidadesTrainSentence= getCombinacaoEntidadesSentence(dic_sentencesTrain, False, '', 0)
print('\n--Dev--')
combinacaoEntidadesDevSentence = getCombinacaoEntidadesSentence(dic_sentencesDev, False, '', 0)
print('\n--Test--')
combinacaoEntidadesTestSentence = getCombinacaoEntidadesSentence(dic_sentencesTest, False, '', 0)


--Train--
Sentence Pairs - Sem filtro-postagger
Sentence Pairs - Sem taxa de Downsampling
key: 999
erro_corpus: 0
num_frases_sem_entidade: 237
len(combinacaoEntidades:) 9696

--Dev--
Sentence Pairs - Sem filtro-postagger
Sentence Pairs - Sem taxa de Downsampling
erro_corpus: 0
num_frases_sem_entidade: 89
len(combinacaoEntidades:) 2159

--Test--
Sentence Pairs - Sem filtro-postagger
Sentence Pairs - Sem taxa de Downsampling
erro_corpus: 0
num_frases_sem_entidade: 107
len(combinacaoEntidades:) 3233


In [34]:
def gravarArquivosTreinamentoSentence(path, combinacaoEntidadesTrain, combinacaoEntidadesDev, combinacaoEntidadesTest):

    # já ir gravando arquivos treinamento, test e dev...
    # pra fazer um teste sem descontinuas
    numTotalEntidades=0
    numTotalEntidadesTrain=0
    numTotalEntidadesDev=0
    numTotalEntidadesTest=0

    existeDir = os.path.exists(path)
    if not existeDir:
        os.makedirs(path)
        
    f_train = open(path+r'\sentence_pairs.train', 'w', encoding='utf-8')

    for i, combinacaoEntidades in enumerate(combinacaoEntidadesTrain):
        if len(combinacaoEntidades)>0:
            termo = combinacaoEntidades[0]
            frase = combinacaoEntidades[1]
            label = str(combinacaoEntidades[2])
            f_train.write(termo+'\t'+frase+'\t'+label+'\n')
            numTotalEntidades=numTotalEntidades+1
            numTotalEntidadesTrain=numTotalEntidadesTrain+1
    f_train.close()

    f_dev = open(path+r'\sentence_pairs.dev', 'w', encoding='utf-8')

    for i, combinacaoEntidades in enumerate(combinacaoEntidadesDev):
        if len(combinacaoEntidades)>0:
            termo = combinacaoEntidades[0]
            frase = combinacaoEntidades[1]
            label = str(combinacaoEntidades[2])
            f_dev.write(termo+'\t'+frase+'\t'+label+'\n')
            numTotalEntidades=numTotalEntidades+1
            numTotalEntidadesDev=numTotalEntidadesDev+1
    f_dev.close()

    f_test = open(path+r'\sentence_pairs.test', 'w', encoding='utf-8')
    for i, combinacaoEntidades in enumerate(combinacaoEntidadesTest):
        #print(dicSentences[i])
        if len(combinacaoEntidades)>0:
            termo = combinacaoEntidades[0]
            frase = combinacaoEntidades[1]
            label = str(combinacaoEntidades[2])
            f_test.write(termo+'\t'+frase+'\t'+label+'\n')
            numTotalEntidades=numTotalEntidades+1
            numTotalEntidadesTest=numTotalEntidadesTest+1
    f_test.close()

    print('numTotalEntidades:', numTotalEntidades)
    print('numTotalEntidadesTrain:', numTotalEntidadesTrain)
    print('numTotalEntidadesDev:', numTotalEntidadesDev)
    print('numTotalEntidadesTest:', numTotalEntidadesTest)


In [30]:
gravarArquivosTreinamentoSentence('sentence-pairs-all',combinacaoEntidadesTrainSentence, combinacaoEntidadesDevSentence, combinacaoEntidadesTestSentence)

numTotalEntidades: 15088
numTotalEntidadesTrain: 9696
numTotalEntidadesDev: 2159
numTotalEntidadesTest: 3233


In [41]:
print('--Train--')
combinacaoEntidadesTrainSentencePos= getCombinacaoEntidadesSentencePos(dic_sentencesTrain)
print('\n--Dev--')
combinacaoEntidadesDevSentencePos = getCombinacaoEntidadesSentencePos(dic_sentencesDev)
print('\n--Test--')
combinacaoEntidadesTestSentencePos = getCombinacaoEntidadesSentencePos(dic_sentencesTest)

gravarArquivosTreinamentoSentence('sentence-pairs-positivos',combinacaoEntidadesTrainSentencePos, combinacaoEntidadesDevSentencePos, combinacaoEntidadesTestSentencePos)

--Train--
Sentence Pairs - So positivos
key: 999
erro_corpus: 0
num_frases_sem_entidade: 237
len(combinacaoEntidades:) 2510

--Dev--
Sentence Pairs - So positivos
erro_corpus: 0
num_frases_sem_entidade: 89
len(combinacaoEntidades:) 703

--Test--
Sentence Pairs - So positivos
erro_corpus: 0
num_frases_sem_entidade: 107
len(combinacaoEntidades:) 991
numTotalEntidades: 4204
numTotalEntidadesTrain: 2510
numTotalEntidadesDev: 703
numTotalEntidadesTest: 991


In [33]:
print('--Train--')
combinacaoEntidadesTrainSentence= getCombinacaoEntidadesSentence(dic_sentencesTrain, True, dicPosTagger, 1)
print('\n--Dev--')
combinacaoEntidadesDevSentence = getCombinacaoEntidadesSentence(dic_sentencesDev, True, dicPosTagger, 1)
print('\n--Test--')
combinacaoEntidadesTestSentence = getCombinacaoEntidadesSentence(dic_sentencesTest, True, dicPosTagger, 1)
gravarArquivosTreinamentoSentence('sentence-pairs-downsampling',combinacaoEntidadesTrainSentence, combinacaoEntidadesDevSentence, combinacaoEntidadesTestSentence)

--Train--
Sentence Pairs - Com filtro-postagger
Sentence Pairs - Com taxa de Downsampling de  1
key: 999
erro_corpus: 0
num_frases_sem_entidade: 237
len(combinacaoEntidades:) 4062

--Dev--
Sentence Pairs - Com filtro-postagger
Sentence Pairs - Com taxa de Downsampling de  1
erro_corpus: 0
num_frases_sem_entidade: 89
len(combinacaoEntidades:) 1048

--Test--
Sentence Pairs - Com filtro-postagger
Sentence Pairs - Com taxa de Downsampling de  1
erro_corpus: 0
num_frases_sem_entidade: 107
len(combinacaoEntidades:) 1555
numTotalEntidades: 6665
numTotalEntidadesTrain: 4062
numTotalEntidadesDev: 1048
numTotalEntidadesTest: 1555


In [132]:
#OLD
print('--Train--')
combinacaoEntidadesTrainSentence= getCombinacaoEntidadesSentence(dic_sentencesTrain, True, dicPosTagger, 1)
print('\n--Dev--')
combinacaoEntidadesDevSentence = getCombinacaoEntidadesSentence(dic_sentencesDev, True, dicPosTagger, 1)
print('\n--Test--')
combinacaoEntidadesTestSentence = getCombinacaoEntidadesSentence(dic_sentencesTest, True, dicPosTagger, 1)


--Train--
Sentence Pairs - Com filtro-postagger
Sentence Pairs - Com taxa de Downsampling de  1
key: 999
erro_corpus: 0
num_frases_sem_entidade: 237
len(combinacaoEntidades:) 5037

--Dev--
Sentence Pairs - Com filtro-postagger
Sentence Pairs - Com taxa de Downsampling de  1
erro_corpus: 0
num_frases_sem_entidade: 89
len(combinacaoEntidades:) 1264

--Test--
Sentence Pairs - Com filtro-postagger
Sentence Pairs - Com taxa de Downsampling de  1
erro_corpus: 0
num_frases_sem_entidade: 107
len(combinacaoEntidades:) 1917


In [136]:
#gravarArquivosTreinamentoSentence('sentence-pairs-filtro',combinacaoEntidadesTrainSentence, combinacaoEntidadesDevSentence, combinacaoEntidadesTestSentence)

numTotalEntidades: 8218
numTotalEntidadesTrain: 5037
numTotalEntidadesDev: 1264
numTotalEntidadesTest: 1917


In [34]:
print('--Train--')
combinacaoEntidadesTrainSentence= getCombinacaoEntidadesSentence(dic_sentencesTrain, True, dicPosTagger, 0)
print('\n--Dev--')
combinacaoEntidadesDevSentence = getCombinacaoEntidadesSentence(dic_sentencesDev, True, dicPosTagger, 0)
print('\n--Test--')
combinacaoEntidadesTestSentence = getCombinacaoEntidadesSentence(dic_sentencesTest, True, dicPosTagger, 0)
gravarArquivosTreinamentoSentence('sentence-pairs-filtro',combinacaoEntidadesTrainSentence, combinacaoEntidadesDevSentence, combinacaoEntidadesTestSentence)

--Train--
Sentence Pairs - Com filtro-postagger
Sentence Pairs - Sem taxa de Downsampling
key: 999
erro_corpus: 0
num_frases_sem_entidade: 237
len(combinacaoEntidades:) 6753

--Dev--
Sentence Pairs - Com filtro-postagger
Sentence Pairs - Sem taxa de Downsampling
erro_corpus: 0
num_frases_sem_entidade: 89
len(combinacaoEntidades:) 1635

--Test--
Sentence Pairs - Com filtro-postagger
Sentence Pairs - Sem taxa de Downsampling
erro_corpus: 0
num_frases_sem_entidade: 107
len(combinacaoEntidades:) 2478
numTotalEntidades: 10866
numTotalEntidadesTrain: 6753
numTotalEntidadesDev: 1635
numTotalEntidadesTest: 2478


In [137]:
'''
print('--Train--')
combinacaoEntidadesTrainSentence= getCombinacaoEntidadesSentence(dic_sentencesTrain, True, dicPosTagger, 0)
print('\n--Dev--')
combinacaoEntidadesDevSentence = getCombinacaoEntidadesSentence(dic_sentencesDev, True, dicPosTagger, 0)
print('\n--Test--')
combinacaoEntidadesTestSentence = getCombinacaoEntidadesSentence(dic_sentencesTest, True, dicPosTagger, 0)
gravarArquivosTreinamentoSentence('sentence-pairs-downsampling',combinacaoEntidadesTrainSentence, combinacaoEntidadesDevSentence, combinacaoEntidadesTestSentence)
'''

--Train--
Sentence Pairs - Com filtro-postagger
Sentence Pairs - Sem taxa de Downsampling
key: 999
erro_corpus: 0
num_frases_sem_entidade: 237
len(combinacaoEntidades:) 6753

--Dev--
Sentence Pairs - Com filtro-postagger
Sentence Pairs - Sem taxa de Downsampling
erro_corpus: 0
num_frases_sem_entidade: 89
len(combinacaoEntidades:) 1635

--Test--
Sentence Pairs - Com filtro-postagger
Sentence Pairs - Sem taxa de Downsampling
erro_corpus: 0
num_frases_sem_entidade: 107
len(combinacaoEntidades:) 2478
numTotalEntidades: 10866
numTotalEntidadesTrain: 6753
numTotalEntidadesDev: 1635
numTotalEntidadesTest: 2478


In [91]:
combinacaoEntidadesTestSentence[0][2]

2

### parte 2.2 - tentar tirar verbos, CC, etc do O para nao ficar com mto FP
### depois, com nosso pos tagger

In [22]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("pucpr-br/postagger-bio-portuguese")
tokenizer = AutoTokenizer.from_pretrained('pucpr-br/postagger-bio-portuguese')

nlp_token_class = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy='max')

def getDicPosTagger(dic_sentencesTrainDev):
    dicPostagger = load_obj('dic_postagger')
    allFrases = load_obj('allFrases')
    if dicPostagger==None or allFrases==None:
        dicPostagger = {}
        allFrases=[]
        for key, value in dic_sentencesTrainDev.items():
            tokens=value[0]
            frase = [t[0] for t in tokens]
            frase = ' '.join(frase)
            allFrases.append(frase)
            #print(frase)

        #print(allFrases)
        doc = nlp_token_class(allFrases)
        #print(doc)
        for frase in doc:
            for d in frase:
                #print(d)
                pos = d['entity_group']
                #print(pos)
                token=d['word']
                if pos=='PREP+ART':
                    pos='ART'
                if pos=='NPROP':
                    pos='N'
                if 'ADV' in pos: # ADV-KS, ADV-KS-REL	
                    pos='ADV'
                if 'PRON' in pos: # PRO-KS	, PRO-KS-REL, PROPESS, PROPSUB
                    pos='PRON'
                if pos=='VAUX'or pos=='PCP': #participio
                    pos='V'
                dicPostagger[token] = pos    

        save_obj('dicPostagger', dicPostagger)
        save_obj('allFrases', allFrases)
    return dicPostagger, allFrases

dic_sentencesTrainDev = load_obj('dic_sentencesTrainDev')

dicPosTagger, _ = getDicPosTagger(dic_sentencesTrainDev)
#dicPosTagger, _ = getDicPosTagger(dic_sentencesTest)
len(dicPosTagger)

3484

In [23]:
def tipoPostaggerTokens(entidade_token, dicPostagger):
    postagger = ''
    for p in entidade_token:
        #print('p:', p)
        if p.lower() in dicPostagger.keys():
            postagger = postagger + '-' + dicPostagger.get(p.lower())
        else:
            #print('nao tem:', p)
            # se nao tem, considera N
            postagger = postagger + '-' + 'N'
    return postagger
tipoPostaggerTokens(['ola','como','aos','você','hoje', '?'], dicPosTagger)

'-N-PREP-ART-N-ADV-PU'

In [24]:
dic_sentencesTrainDev = load_obj('dic_sentencesTrainDev')

In [25]:
def getListaPostaggerEntidades(dic_sentencesTrainDev, dicPosTagger):
    lista_postaggers_entidades = []
    for key, value in dic_sentencesTrainDev.items():
        entidades = value[1]
        for entidade in entidades:
            #print(entidade[0])
            pos_tagger=tipoPostaggerTokens(entidade[0].split(), dicPosTagger)
            if pos_tagger not in lista_postaggers_entidades:
                lista_postaggers_entidades.append(pos_tagger)
    return lista_postaggers_entidades

lista_postaggers_entidades = getListaPostaggerEntidades(dic_sentencesTrainDev, dicPosTagger)
lista_postaggers_entidades[:10]

['-N-ADJ-ART-N',
 '-N-N-N-ART-N-ART-N',
 '-N',
 '-N-ADJ',
 '-ADJ',
 '-V',
 '-N-NUM',
 '-N-PU-ADJ-N',
 '-ADJ-N-N-ADJ-PREP-N',
 '-N-PREP-N-ART-N']

In [26]:
lista_postaggers_entidades[-10:-1]

['-N-ADJ-ADJ-N-N-N',
 '-N-ADJ-N-N-N',
 '-ADJ-N-N-N',
 '-N-ADV-N-N-PU-NUM-N',
 '-ADJ-N-PREP-N',
 '-N-PREP-N-N-ADV',
 '-V-PDEN-PREP-ADJ-N',
 '-N-N-PREP-N-PU-N-PU-N-PU-N-PREP-N',
 '-ADJ-N-PREP-N-ADJ']

In [77]:
print('-KC' in lista_postaggers_entidades) # e
print('-PREP+ART' in lista_postaggers_entidades) 
print('-PREP' in lista_postaggers_entidades)
print('-ART' in lista_postaggers_entidades)
print('-V' in lista_postaggers_entidades)
print('-ADJ' in lista_postaggers_entidades)
print('-PU' in lista_postaggers_entidades)
print('-PCP' in lista_postaggers_entidades)

False
False
True
True
True
True
False
False


In [71]:
print('--Train--')
combinacaoEntidadesTrain, pulando_termos_postaggerTrain = getCombinacaoEntidades(dic_sentencesTrain, True, dicPosTagger, 0)
print('\n--Dev--')
combinacaoEntidadesDev, pulando_termos_postaggerDev= getCombinacaoEntidades(dic_sentencesDev, True, dicPosTagger, 0)
print('\n--Test--')
combinacaoEntidadesTest, pulando_termos_postaggerTest= getCombinacaoEntidades(dic_sentencesTest, True, dicPosTagger, 0)

--Train--
Com filtro-postagger
Sem taxa de Downsampling
key: 999
erro_corpus: 0
num_frases_sem_entidade: 237
len(combinacaoEntidadesAll:) 1319
len(pulando_termos_postagger): 3925

--Dev--
Com filtro-postagger
Sem taxa de Downsampling
erro_corpus: 0
num_frases_sem_entidade: 89
len(combinacaoEntidadesAll:) 416
len(pulando_termos_postagger): 745

--Test--
Com filtro-postagger
Sem taxa de Downsampling
erro_corpus: 0
num_frases_sem_entidade: 107
len(combinacaoEntidadesAll:) 506
len(pulando_termos_postagger): 1183


In [72]:
combinacaoEntidadesTest[14]

['aumento moderado de átrio esquerdo .',
 [['aumento moderado <e1> de átrio esquerdo </e1> .', 'O'],
  ['aumento <e1> moderado </e1> de átrio esquerdo .', 'O'],
  ['aumento moderado <e1> de </e1> átrio esquerdo .', 'O'],
  ['aumento <e1> moderado de </e1> átrio esquerdo .', 'O'],
  ['<e1> aumento moderado de átrio esquerdo </e1> .', 'Problema'],
  ['aumento moderado de <e1> átrio esquerdo </e1> .', 'Anatomia'],
  ['aumento moderado de átrio <e1> esquerdo </e1> .', 'O'],
  ['aumento <e1> moderado de átrio esquerdo </e1> .', 'O'],
  ['aumento moderado <e1> de átrio </e1> esquerdo .', 'O'],
  ['aumento <e1> moderado de átrio </e1> esquerdo .', 'O'],
  ['<e1> aumento </e1> moderado de átrio esquerdo .', 'O'],
  ['aumento moderado de <e1> átrio </e1> esquerdo .', 'O']]]

In [73]:
pulando_termos_postaggerTest[:50]

[[['25'], '-NUM'],
 [['ventrículo', 'esquerdo', 'com'], '-N-ADJ-PREP'],
 [['ventrículo', 'esquerdo', 'com', 'hipertrofia', 'concentrica', 'de'],
  '-N-ADJ-PREP-N-N-PREP'],
 [['ventrículo',
   'esquerdo',
   'com',
   'hipertrofia',
   'concentrica',
   'de',
   'grau'],
  '-N-ADJ-PREP-N-N-PREP-N'],
 [['ventrículo',
   'esquerdo',
   'com',
   'hipertrofia',
   'concentrica',
   'de',
   'grau',
   'discreto'],
  '-N-ADJ-PREP-N-N-PREP-N-ADJ'],
 [['esquerdo', 'com'], '-ADJ-PREP'],
 [['esquerdo', 'com', 'hipertrofia'], '-ADJ-PREP-N'],
 [['esquerdo', 'com', 'hipertrofia', 'concentrica'], '-ADJ-PREP-N-N'],
 [['esquerdo', 'com', 'hipertrofia', 'concentrica', 'de'],
  '-ADJ-PREP-N-N-PREP'],
 [['esquerdo', 'com', 'hipertrofia', 'concentrica', 'de', 'grau'],
  '-ADJ-PREP-N-N-PREP-N'],
 [['esquerdo', 'com', 'hipertrofia', 'concentrica', 'de', 'grau', 'discreto'],
  '-ADJ-PREP-N-N-PREP-N-ADJ'],
 [['com', 'hipertrofia', 'concentrica', 'de'], '-PREP-N-N-PREP'],
 [['com', 'hipertrofia', 'concentrica

In [74]:
pulando_termos_postaggerTrain[:20]

[[['Dispneia', 'importante', 'aos'], '-N-ADJ-ART'],
 [['importante', 'aos'], '-ADJ-ART'],
 [['importante', 'aos', 'esforços'], '-ADJ-ART-N'],
 [['aos', 'esforços'], '-ART-N'],
 [['dor', 'tipo', 'peso', 'no'], '-N-N-N-ART'],
 [['dor', 'tipo', 'peso', 'no', 'peito'], '-N-N-N-ART-N'],
 [['dor', 'tipo', 'peso', 'no', 'peito', 'no'], '-N-N-N-ART-N-ART'],
 [['tipo', 'peso', 'no'], '-N-N-ART'],
 [['tipo', 'peso', 'no', 'peito', 'no'], '-N-N-ART-N-ART'],
 [['tipo', 'peso', 'no', 'peito', 'no', 'esforço'], '-N-N-ART-N-ART-N'],
 [['peso', 'no', 'peito', 'no'], '-N-ART-N-ART'],
 [['no', 'peito'], '-ART-N'],
 [['no', 'peito', 'no'], '-ART-N-ART'],
 [['no', 'peito', 'no', 'esforço'], '-ART-N-ART-N'],
 [['no', 'esforço'], '-ART-N'],
 [['100'], '-NUM'],
 [['25'], '-NUM'],
 [['175'], '-NUM'],
 [['40'], '-NUM'],
 [['20'], '-NUM']]

In [75]:
gravarArquivosTreinamento('com_filtro',combinacaoEntidadesTrain, combinacaoEntidadesDev, combinacaoEntidadesTest)

numTotalEntidades: 10626
numTotalEntidadesTrain: 6578
numTotalEntidadesDev: 1611
numTotalEntidadesTest: 2437


In [67]:
#OLD
#gravarArquivosTreinamento('com_filtro',combinacaoEntidadesTrain, combinacaoEntidadesDev, combinacaoEntidadesTest)

numTotalEntidades: 8965
numTotalEntidadesTrain: 5533
numTotalEntidadesDev: 1389
numTotalEntidadesTest: 2043


In [76]:
# com downsampling
print('--Train--')
combinacaoEntidadesTrain, pulando_termos_postaggerTrain = getCombinacaoEntidades(dic_sentencesTrain, True, dicPosTagger, 1)
print('\n--Dev--')
combinacaoEntidadesDev, pulando_termos_postaggerDev= getCombinacaoEntidades(dic_sentencesDev, True, dicPosTagger, 1)
print('\n--Test--')
combinacaoEntidadesTest, pulando_termos_postaggerTest= getCombinacaoEntidades(dic_sentencesTest, True, dicPosTagger, 1)

gravarArquivosTreinamento('com-filtro-downsampling',combinacaoEntidadesTrain, combinacaoEntidadesDev, combinacaoEntidadesTest)

--Train--
Com filtro-postagger
Com taxa de Downsampling de  1
key: 999
erro_corpus: 0
num_frases_sem_entidade: 237
len(combinacaoEntidadesAll:) 1319
len(pulando_termos_postagger): 3925

--Dev--
Com filtro-postagger
Com taxa de Downsampling de  1
erro_corpus: 0
num_frases_sem_entidade: 89
len(combinacaoEntidadesAll:) 416
len(pulando_termos_postagger): 745

--Test--
Com filtro-postagger
Com taxa de Downsampling de  1
erro_corpus: 0
num_frases_sem_entidade: 107
len(combinacaoEntidadesAll:) 506
len(pulando_termos_postagger): 1183
numTotalEntidades: 6664
numTotalEntidadesTrain: 4055
numTotalEntidadesDev: 1050
numTotalEntidadesTest: 1559


### Agora, tratar descontinuas

In [837]:
len(combinacaoEntidadesAll_desc)

1568

In [838]:
len(combinacaoEntidadesAll)

16614

In [840]:
# já ir gravando arquivos treinamento, test e dev...
# pra fazer um teste sem descontinuas
numTreinamento = len(combinacaoEntidadesAll)*0.8
numTreinamento = int(numTreinamento)
numTeste = (len(combinacaoEntidadesAll) - numTreinamento)/2
numTeste = int(numTeste)
numTotalEntidades=0
numTotalEntidadesTrain=0
numTotalEntidadesDev=0
numTotalEntidadesTest=0

print('numTreinamento:', numTreinamento)
print('numTeste:', numTeste)

f_train = open('genia.train', 'w')
f_dev = open('genia.dev', 'w')
f_test = open('genia.test', 'w')
f_dev_entidades = open('genia_entidades.dev', 'w')
f_test_entidades = open('genia_entidades.test', 'w')

for i, combinacaoEntidades in enumerate(combinacaoEntidadesAll):
    #print(dicSentences[i])
    frase = combinacaoEntidades[0]
    frases_entidade = combinacaoEntidades[1]
    if i<=numTreinamento:
        f_train.write(frase+'\n')
        for frase_entidade in frases_entidade:
            f_train.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
            numTotalEntidades=numTotalEntidades+1
            numTotalEntidadesTrain=numTotalEntidadesTrain+1
    elif i <= numTreinamento + numTeste:
        f_dev.write(frase+'\n')
        for frase_entidade in frases_entidade:
            f_dev.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
            f_dev_entidades.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
            numTotalEntidades=numTotalEntidades+1
            numTotalEntidadesDev=numTotalEntidadesDev+1
    else:
        f_test.write(frase+'\n')
        for frase_entidade in frases_entidade:
            f_test.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
            f_test_entidades.write(frase_entidade[1]+'\t'+frase_entidade[0]+'\n')
            numTotalEntidades=numTotalEntidades+1
            numTotalEntidadesTest=numTotalEntidadesTest+1

f_train.close()
f_test.close()
f_dev.close()
f_test_entidades.close()
f_dev_entidades.close()

print('numTotalEntidades:', numTotalEntidades)
print('numTotalEntidadesTrain:', numTotalEntidadesTrain)
print('numTotalEntidadesDev:', numTotalEntidadesDev)
print('numTotalEntidadesTest:', numTotalEntidadesTest)


numTreinamento: 13291
numTeste: 1661
numTotalEntidades: 165571
numTotalEntidadesTrain: 131376
numTotalEntidadesDev: 16986
numTotalEntidadesTest: 17209
